In [ ]:
!git clone https://github.com/ShubhayanS/Multiclass-Diabetic-Retinopathy-Detection.git

Cloning into 'Multiclass-Diabetic-Retinopathy-Detection'...
remote: Enumerating objects: 440, done.
remote: Total 440 (delta 0), reused 0 (delta 0), pack-reused 440 (from 1)
Receiving objects: 100% (440/440), 29.00 MiB | 24.38 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [ ]:
%cd /content/Multiclass-Diabetic-Retinopathy-Detection

/content/Multiclass-Diabetic-Retinopathy-Detection


In [ ]:
!pwd

/content/Multiclass-Diabetic-Retinopathy-Detection


## Install independences

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
import sys
import tensorflow as tf
import time
import re
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from tensorflow.core.framework import attr_value_pb2
from tensorflow.core.framework import graph_pb2
from tensorflow.core.framework import node_def_pb2
from tensorflow.python.client import session
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import importer
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.framework import tensor_util
from tensorflow.python.ops import array_ops
from tensorflow.python.platform import app
from tensorflow.python.platform import flags as flags_lib
from tensorflow.python.platform import gfile
from IPython.display import Image, HTML, display

import argparse
import collections
from datetime import datetime
import hashlib
import os.path
import random
import tarfile

import numpy as np
from six.moves import urllib
import tensorflow as tf
from tensorflow.python.util import compat
import PIL.Image as Image

Instructions for updating:
non-resource variables are not supported in the long term


## count_ops.py
Đếm số lượng phép toán trong mô hình
- Kiểm tra nhanh cấu trúc mô hình
- Đếm tổng số phép toán hoặc đếm theo tên cụ thể

In [ ]:
def load_graph(file_name):
    with open(file_name, 'rb') as f:
        content = f.read()
    graph_def = tf.GraphDef()  # Khởi tạo đồ thị trống
    graph_def.ParseFromString(content)  # Parse nội dung nhị phân thành đồ thị TensorFlow
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')  # Nhập đồ thị
    return graph  # Trả về graph để thao tác tiếp

def count_ops(file_name, op_name=None):
    graph = load_graph(file_name)
    if op_name is None:
        return len(graph.get_operations())  # Trả về tổng số phép toán trong mô hình
    else:
        return sum(1 for op in graph.get_operations() if op.name == op_name)# Trả về số lượng phép toán có tên đúng bằng op_name

# if __name__ == "__main__":
#   os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#   print(count_ops(*sys.argv[1:]))

## show_image.py

In [ ]:
root = "tf_files/flower_photos/"
with open(root+"/LICENSE.txt") as f:
    attributions = f.readlines()[4:]
attributions = [line.split(' CC-BY') for line in attributions]
attributions = dict(attributions)

def show_image(image_path):
    display(Image(image_path))

    image_rel = image_path.replace(root,'')
    caption = "Image " + ' - '.join(attributions[image_rel].split(' - ')[:-1])
    display(HTML("<div>%s</div>" % caption))

## label_image.py

In [ ]:
def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.GraphDef()
  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)
  return graph

def read_tensor_from_image_file(file_name, input_height=299, input_width=299,input_mean=0, input_std=255):
  # Đọc ảnh từ file, resize, chuẩn hóa giá trị về [-1, 1] hoặc [0, 1] tùy input_mean, input_std
  # Trả về tensor ảnh đã sẵn sàng để đưa vào mô hình
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(file_reader, channels = 3, name='png_reader')
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(tf.image.decode_gif(file_reader, name='gif_reader'))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
  else:
    image_reader = tf.image.decode_jpeg(file_reader, channels = 3,name='jpeg_reader')
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0);
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.Session()
  result = sess.run(normalized)
  return result

def load_labels(label_file): # # Trả về danh sách nhãn theo thứ tự lớp trong mô hình
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label

if __name__ == "__main__": # giá trị mặc định
  file_name = "tf_files/flower_photos/daisy/3475870145_685a19116d.jpg"
  model_file = "tf_files/retrained_graph.pb"
  label_file = "tf_files/retrained_labels.txt"
  input_height = 224
  input_width = 224
  input_mean = 128
  input_std = 128
  input_layer = "input"
  output_layer = "final_result"

  parser = argparse.ArgumentParser()
  parser.add_argument("--image", help="image to be processed")
  parser.add_argument("--graph", help="graph/model to be executed")
  parser.add_argument("--labels", help="name of file containing labels")
  parser.add_argument("--input_height", type=int, help="input height")
  parser.add_argument("--input_width", type=int, help="input width")
  parser.add_argument("--input_mean", type=int, help="input mean")
  parser.add_argument("--input_std", type=int, help="input std")
  parser.add_argument("--input_layer", help="name of input layer")
  parser.add_argument("--output_layer", help="name of output layer")
  args = parser.parse_args()

  if args.graph:
    model_file = args.graph
  if args.image:
    file_name = args.image
  if args.labels:
    label_file = args.labels
  if args.input_height:
    input_height = args.input_height
  if args.input_width:
    input_width = args.input_width
  if args.input_mean:
    input_mean = args.input_mean
  if args.input_std:
    input_std = args.input_std
  if args.input_layer:
    input_layer = args.input_layer
  if args.output_layer:
    output_layer = args.output_layer

  graph = load_graph(model_file)
  # Đọc và xử lý ảnh đầu vào
  t = read_tensor_from_image_file(file_name,
                                  input_height=input_height,
                                  input_width=input_width,
                                  input_mean=input_mean,
                                  input_std=input_std)

  input_name = "import/" + input_layer
  output_name = "import/" + output_layer
  input_operation = graph.get_operation_by_name(input_name);
  output_operation = graph.get_operation_by_name(output_name);

  with tf.Session(graph=graph) as sess: # Chạy mô hình và tính thời gian:
    start = time.time()
    results = sess.run(output_operation.outputs[0],{input_operation.outputs[0]: t}) #  Nhận kết quả dự đoán (vector xác suất từng lớp)
    end=time.time()
  results = np.squeeze(results)  # Bỏ chiều dư thừa
  top_k = results.argsort()[-5:][::-1]  # Lấy top 5 kết quả xác suất cao nhất
  labels = load_labels(label_file)  # Đọc danh sách nhãn

  print('\nEvaluation time (1-image): {:.3f}s\n'.format(end - start))
  template = "{} (score={:0.5f})"
  for i in top_k:
      print(template.format(labels[i], results[i]))


## retrain.py
- Tận dụng mô hình đã huấn luyện sẵn (Inception) trên ImageNet
- Tái huấn luyện lớp cuối cùng (softmax) cho bài toán phân loại ảnh của bạn
- Hỗ trợ tăng tốc bằng cache "bottleneck" (vector đặc trưng trung gian)
- Có thể sinh log TensorBoard để theo dõi huấn luyện

Trong mạng nơ-ron sâu (deep neural network), "bottleneck" là:
  - Vector đặc trưng trung gian
  - Được trích xuất từ mô hình pre-trained (như InceptionV3, MobileNet)
  - Thường nằm ở lớp ngay trước softmax cuối cùng

Bản chất:
  - Bottleneck chứa toàn bộ thông tin đặc trưng cần thiết của ảnh
  - Được xem như "đại diện nén" của ảnh đầu vào
  
VD
- Ảnh võng mạc đầu vào:
```
  Ảnh: retina_001.jpg
  Kích thước: 512 x 512 x 3 (RGB)
  ```
- Sau khi đi qua mô hình gốc Inception:
```
Bottleneck output: Vector có kích thước (1, 2048)
[0.12, -0.55, 0.08, 1.23, ..., 0.04]  # 2048 giá trị float, giá trị bottleneck (đã nén thông tin quan trọng của ảnh)
```
- Đây là "đại diện nén" cho ảnh võng mạc, chứa thông tin về:
  - Mức độ tổn thương võng mạc
  - Mạch máu, vi điểm xuất huyết, vùng tổn thương

In [ ]:
FLAGS = None
# Đây là tất cả các tham số gắn liền với kiến trúc mô hình cụ thể mà chúng ta đang sử dụng là Inception v3. Các tham số này bao gồm những thứ như tên  và kích thước của tensor. Nếu bạn muốn điều chỉnh tập lệnh (script) này để hoạt động với một mô hình khác, bạn sẽ cần cập nhật chúng để phản ánh các giá trị trong mạng (neuron) mà bạn đang sử dụng.

MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1  # ~134M

# Quét thư mục ảnh, tự chia train/val/test
def create_image_lists(image_dir, testing_percentage, validation_percentage):
  """Xây dựng một danh sách các ảnh huấn luyện từ hệ thống tệp.

 Phân tích các thư mục con trong thư mục chứa ảnh, chia chúng thành các tập dữ liệu ổn định cho huấn luyện (training), kiểm th (testing) và xác thực (validation), đồng thời trả về một cấu trúc dữ liệu mô tả danh sách các ảnh cho mỗi nhãn và đường dẫn của chúng.

 Tham số:
    image_dir: Đường dẫn dạng chuỗi đến một thư mục chứa các thư mục con của ảnh.
    testing_percentage: Tỷ lệ phần trăm (số nguyên) của ảnh được dành riêng cho việc kiểm thử.
    validation_percentage: Tỷ lệ phần trăm (số nguyên) của ảnh được dành riêng cho việc xác thực.

 Trả về:
    Một dictionary chứa một mục cho mỗi thư mục con của nhãn, với các ảnh được chia thành các tập huấn luyện, kiểm thử và xác thực bên trong mỗi nhãn.
 """
  if not gfile.Exists(image_dir):
    tf.logging.error("Image directory '" + image_dir + "' not found.")
    return None
  result = collections.OrderedDict()
  sub_dirs = [
    os.path.join(image_dir,item)
    for item in gfile.ListDirectory(image_dir)]
  sub_dirs = sorted(item for item in sub_dirs
                    if gfile.IsDirectory(item))
  for sub_dir in sub_dirs:
    extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']
    file_list = []
    dir_name = os.path.basename(sub_dir)
    if dir_name == image_dir:
      continue
    tf.logging.info("Looking for images in '" + dir_name + "'")
    for extension in extensions:
      file_glob = os.path.join(image_dir, dir_name, '*.' + extension)
      file_list.extend(gfile.Glob(file_glob))
    if not file_list:
      tf.logging.warning('No files found')
      continue
    # Cảnh báo nếu lớp quá ít hoặc quá nhiều ảnh.
    if len(file_list) < 20:
      tf.logging.warning(
          'WARNING: Folder has less than 20 images, which may cause issues.')
    elif len(file_list) > MAX_NUM_IMAGES_PER_CLASS:
      tf.logging.warning(
          'WARNING: Folder {} has more than {} images. Some images will never be selected.'.format(dir_name, MAX_NUM_IMAGES_PER_CLASS))
    label_name = re.sub(r'[^a-z0-9]+', ' ', dir_name.lower())
    training_images = []
    testing_images = []
    validation_images = []
    for file_name in file_list:
      base_name = os.path.basename(file_name)
      # Mục đích của đoạn code dưới này là để đảm bảo rằng các ảnh rất giống nhau (ví dụ: nhiều ảnh chụp cùng một chiếc lá ở các góc độ hơi khác nhau) sẽ luôn được xếp vào cùng một tập dữ liệu (cùng vào tập huấn luyện, hoặc cùng vào tập kiểm thử).

      hash_name = re.sub(r'_nohash_.*$', '', file_name) # Loại bỏ phần _nohash_ nếu có để nhóm các ảnh giống nhau lại
      hash_name_hashed = hashlib.sha1(compat.as_bytes(hash_name)).hexdigest() # Tạo mã băm (hash) ổn định từ tên ảnh
      # Chuyển mã băm thành giá trị phần trăm [0, 100]
      percentage_hash = ((int(hash_name_hashed, 16) %  (MAX_NUM_IMAGES_PER_CLASS + 1)) * (100.0 / MAX_NUM_IMAGES_PER_CLASS))

      # Ảnh được phân chia dựa vào phần trăm hash:
      # < validation_percentage → vào tập validation
      # < testing_percentage + validation_percentage → vào tập testing
      # Còn lại vào tập training
      if percentage_hash < validation_percentage:
        validation_images.append(base_name)
      elif percentage_hash < (testing_percentage + validation_percentage):
        testing_images.append(base_name)
      else:
        training_images.append(base_name)
    result[label_name] = {
        'dir': dir_name,
        'training': training_images,
        'testing': testing_images,
        'validation': validation_images,
    }
  return result

  # Tên ảnh: "retina_diabetic_001_nohash_3.jpg"
  # Sau khi bỏ "_nohash_": "retina_diabetic_001.jpg"
  # Tính hash ra %: Giả sử là 7.5%
  # Nếu validation_percentage = 10% → Ảnh vào tập validation

def get_image_path(image_lists, label_name, index, image_dir, category):
  """"Returns a path to an image for a label at the given index.

  Args:
    image_lists: Dictionary of training images for each label.
    label_name: Label string we want to get an image for.
    index: Int offset of the image we want. This will be moduloed by the
    available number of images for the label, so it can be arbitrarily large.
    image_dir: Root folder string of the subfolders containing the training
    images.
    category: Name string of set to pull images from - training, testing, or
    validation.

  Returns:
    File system path string to an image that meets the requested parameters.

  """
  if label_name not in image_lists:
    tf.logging.fatal('Label does not exist %s.', label_name)
  label_lists = image_lists[label_name]
  if category not in label_lists:
    tf.logging.fatal('Category does not exist %s.', category)
  category_list = label_lists[category]
  if not category_list:
    tf.logging.fatal('Label %s has no images in the category %s.',
                     label_name, category)
  mod_index = index % len(category_list)
  base_name = category_list[mod_index]
  sub_dir = label_lists['dir']
  full_path = os.path.join(image_dir, sub_dir, base_name)
  return full_path


def get_bottleneck_path(image_lists, label_name, index, bottleneck_dir,
                        category, architecture):
  """"Returns a path to a bottleneck file for a label at the given index.

  Args:
    image_lists: Dictionary of training images for each label.
    label_name: Label string we want to get an image for.
    index: Integer offset of the image we want. This will be moduloed by the
    available number of images for the label, so it can be arbitrarily large.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    category: Name string of set to pull images from - training, testing, or
    validation.
    architecture: The name of the model architecture.

  Returns:
    File system path string to an image that meets the requested parameters.
  """
  return get_image_path(image_lists, label_name, index, bottleneck_dir,
                        category) + '_' + architecture + '.txt'


def create_model_graph(model_info):
  """"Creates a graph from saved GraphDef file and returns a Graph object.

  Args:
    model_info: Dictionary containing information about the model architecture.

  Returns:
    Graph holding the trained Inception network, and various tensors we'll be
    manipulating.
  """
  with tf.Graph().as_default() as graph:
    model_path = os.path.join(FLAGS.model_dir, model_info['model_file_name'])
    with gfile.FastGFile(model_path, 'rb') as f:
      graph_def = tf.GraphDef()
      graph_def.ParseFromString(f.read())
      bottleneck_tensor, resized_input_tensor = (tf.import_graph_def(
          graph_def,
          name='',
          return_elements=[
              model_info['bottleneck_tensor_name'],
              model_info['resized_input_tensor_name'],
          ]))
  return graph, bottleneck_tensor, resized_input_tensor

# Trích xuất vector bottleneck từ ảnh
def run_bottleneck_on_image(sess, image_data, image_data_tensor,
                            decoded_image_tensor, resized_input_tensor,
                            bottleneck_tensor):
  """Runs inference on an image to extract the 'bottleneck' summary layer.

  Args:
    sess: Current active TensorFlow Session.
    image_data: String of raw JPEG data.
    image_data_tensor: Input data layer in the graph.
    decoded_image_tensor: Output of initial image resizing and  preprocessing.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: Layer before the final softmax.

  Returns:
    Numpy array of bottleneck values.
  """
  # First decode the JPEG image, resize it, and rescale the pixel values.
  resized_input_values = sess.run(decoded_image_tensor,
                                  {image_data_tensor: image_data})
  # Then run it through the recognition network.
  bottleneck_values = sess.run(bottleneck_tensor,
                               {resized_input_tensor: resized_input_values})
  bottleneck_values = np.squeeze(bottleneck_values)
  return bottleneck_values

# Tải mô hình pre-trained về
def maybe_download_and_extract(data_url):
  """Download and extract model tar file.

  If the pretrained model we're using doesn't already exist, this function
  downloads it from the TensorFlow.org website and unpacks it into a directory.

  Args:
    data_url: Web location of the tar file containing the pretrained model.
  """
  dest_directory = FLAGS.model_dir
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  filename = data_url.split('/')[-1]
  filepath = os.path.join(dest_directory, filename)
  if not os.path.exists(filepath):

    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' %
                       (filename,
                        float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()

    filepath, _ = urllib.request.urlretrieve(data_url, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    tf.logging.info('Successfully downloaded', filename, statinfo.st_size,
                    'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(dest_directory)


def ensure_dir_exists(dir_name):
  """Makes sure the folder exists on disk.

  Args:
    dir_name: Path string to the folder we want to create.
  """
  if not os.path.exists(dir_name):
    os.makedirs(dir_name)


bottleneck_path_2_bottleneck_values = {}


def create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor):
  """Create a single bottleneck file."""
  tf.logging.info('Creating bottleneck at ' + bottleneck_path)
  image_path = get_image_path(image_lists, label_name, index,
                              image_dir, category)
  if not gfile.Exists(image_path):
    tf.logging.fatal('File does not exist %s', image_path)
  image_data = gfile.FastGFile(image_path, 'rb').read()
  try:
    bottleneck_values = run_bottleneck_on_image(
        sess, image_data, jpeg_data_tensor, decoded_image_tensor,
        resized_input_tensor, bottleneck_tensor)
  except Exception as e:
    raise RuntimeError('Error during processing file %s (%s)' % (image_path,
                                                                 str(e)))
  bottleneck_string = ','.join(str(x) for x in bottleneck_values)
  with open(bottleneck_path, 'w') as bottleneck_file:
    bottleneck_file.write(bottleneck_string)


def get_or_create_bottleneck(sess, image_lists, label_name, index, image_dir,
                             category, bottleneck_dir, jpeg_data_tensor,
                             decoded_image_tensor, resized_input_tensor,
                             bottleneck_tensor, architecture):
  """Retrieves or calculates bottleneck values for an image.

  If a cached version of the bottleneck data exists on-disk, return that,
  otherwise calculate the data and save it to disk for future use.

  Args:
    sess: The current active TensorFlow Session.
    image_lists: Dictionary of training images for each label.
    label_name: Label string we want to get an image for.
    index: Integer offset of the image we want. This will be modulo-ed by the
    available number of images for the label, so it can be arbitrarily large.
    image_dir: Root folder string  of the subfolders containing the training
    images.
    category: Name string of which  set to pull images from - training, testing,
    or validation.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    jpeg_data_tensor: The tensor to feed loaded jpeg data into.
    decoded_image_tensor: The output of decoding and resizing the image.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: The output tensor for the bottleneck values.
    architecture: The name of the model architecture.

  Returns:
    Numpy array of values produced by the bottleneck layer for the image.
  """
  label_lists = image_lists[label_name]
  sub_dir = label_lists['dir']
  sub_dir_path = os.path.join(bottleneck_dir, sub_dir)
  ensure_dir_exists(sub_dir_path)
  bottleneck_path = get_bottleneck_path(image_lists, label_name, index,
                                        bottleneck_dir, category, architecture)
  if not os.path.exists(bottleneck_path):
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor)
  with open(bottleneck_path, 'r') as bottleneck_file:
    bottleneck_string = bottleneck_file.read()
  did_hit_error = False
  try:
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
  except ValueError:
    tf.logging.warning('Invalid float found, recreating bottleneck')
    did_hit_error = True
  if did_hit_error:
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor)
    with open(bottleneck_path, 'r') as bottleneck_file:
      bottleneck_string = bottleneck_file.read()
    # Allow exceptions to propagate here, since they shouldn't happen after a
    # fresh creation
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
  return bottleneck_values

#
def cache_bottlenecks(sess, image_lists, image_dir, bottleneck_dir,
                      jpeg_data_tensor, decoded_image_tensor,
                      resized_input_tensor, bottleneck_tensor, architecture):
  """Đảm bảo rằng tất cả các giá trị bottleneck cho tập huấn luyện, kiểm thử và xác thực đều được lưu vào bộ nhớ đệm (cached).

  Bởi vì chúng ta có khả năng đọc cùng một hình ảnh nhiều lần (nếu không có
  biến dạng nào được áp dụng trong quá trình huấn luyện), việc tính toán giá trị
  của lớp bottleneck một lần cho mỗi ảnh trong giai đoạn tiền xử lý, và sau đó
  chỉ đọc các giá trị đã được lưu trong bộ nhớ đệm nhiều lần trong quá trình huấn luyện
  sẽ giúp tăng tốc rất nhiều. Ở đây, chúng ta sẽ duyệt qua tất cả các hình ảnh
  đã tìm thấy, tính toán các giá trị đó và lưu chúng lại.

  Tham số (Args):
      sess: Session (phiên làm việc) TensorFlow đang hoạt động.
      image_lists: Dictionary chứa các ảnh huấn luyện cho mỗi nhãn.
      image_dir: Chuỗi đường dẫn đến thư mục gốc chứa các thư mục con của ảnh huấn luyện.
      bottleneck_dir: Chuỗi đường dẫn đến thư mục chứa các tệp giá trị bottleneck đã được lưu.
      jpeg_data_tensor: Tensor đầu vào cho dữ liệu jpeg từ tệp.
      decoded_image_tensor: Tensor đầu ra của việc giải mã và thay đổi kích thước ảnh.
      resized_input_tensor: Nút đầu vào của đồ thị nhận dạng.
      bottleneck_tensor: Lớp đầu ra kế cuối (penultimate) của đồ thị.
      architecture: Tên của kiến trúc mô hình.

  Trả về (Returns):
      Không có gì (Nothing).
  """
  how_many_bottlenecks = 0
  ensure_dir_exists(bottleneck_dir)
  for label_name, label_lists in image_lists.items():
    for category in ['training', 'testing', 'validation']:
      category_list = label_lists[category]
      for index, unused_base_name in enumerate(category_list):
        get_or_create_bottleneck(
            sess, image_lists, label_name, index, image_dir, category,
            bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
            resized_input_tensor, bottleneck_tensor, architecture)

        how_many_bottlenecks += 1
        if how_many_bottlenecks % 100 == 0:
          tf.logging.info(
              str(how_many_bottlenecks) + ' bottleneck files created.')

#  Lấy ngẫu nhiên vector đặc trưng từ cache .txt
def get_random_cached_bottlenecks(sess, image_lists, how_many, category,
                                  bottleneck_dir, image_dir, jpeg_data_tensor,
                                  decoded_image_tensor, resized_input_tensor,
                                  bottleneck_tensor, architecture):
  """Retrieves bottleneck values for cached images.

  If no distortions are being applied, this function can retrieve the cached
  bottleneck values directly from disk for images. It picks a random set of
  images from the specified category.

  Args:
    sess: Current TensorFlow Session.
    image_lists: Dictionary of training images for each label.
    how_many: If positive, a random sample of this size will be chosen.
    If negative, all bottlenecks will be retrieved.
    category: Name string of which set to pull from - training, testing, or
    validation.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    image_dir: Root folder string of the subfolders containing the training
    images.
    jpeg_data_tensor: The layer to feed jpeg image data into.
    decoded_image_tensor: The output of decoding and resizing the image.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: The bottleneck output layer of the CNN graph.
    architecture: The name of the model architecture.

  Returns:
    List of bottleneck arrays, their corresponding ground truths, and the
    relevant filenames.
  """
  class_count = len(image_lists.keys())
  bottlenecks = []
  ground_truths = []
  filenames = []
  if how_many >= 0:
    # Retrieve a random sample of bottlenecks.
    for unused_i in range(how_many):
      label_index = random.randrange(class_count)
      label_name = list(image_lists.keys())[label_index]
      image_index = random.randrange(MAX_NUM_IMAGES_PER_CLASS + 1)
      image_name = get_image_path(image_lists, label_name, image_index,
                                  image_dir, category)
      bottleneck = get_or_create_bottleneck(
          sess, image_lists, label_name, image_index, image_dir, category,
          bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
          resized_input_tensor, bottleneck_tensor, architecture)
      ground_truth = np.zeros(class_count, dtype=np.float32)
      ground_truth[label_index] = 1.0
      bottlenecks.append(bottleneck)
      ground_truths.append(ground_truth)
      filenames.append(image_name)
  else:
    # Retrieve all bottlenecks.
    for label_index, label_name in enumerate(image_lists.keys()):
      for image_index, image_name in enumerate(
          image_lists[label_name][category]):
        image_name = get_image_path(image_lists, label_name, image_index,
                                    image_dir, category)
        bottleneck = get_or_create_bottleneck(
            sess, image_lists, label_name, image_index, image_dir, category,
            bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
            resized_input_tensor, bottleneck_tensor, architecture)
        ground_truth = np.zeros(class_count, dtype=np.float32)
        ground_truth[label_index] = 1.0
        bottlenecks.append(bottleneck)
        ground_truths.append(ground_truth)
        filenames.append(image_name)
  return bottlenecks, ground_truths, filenames


def get_random_distorted_bottlenecks(
    sess, image_lists, how_many, category, image_dir, input_jpeg_tensor,
    distorted_image, resized_input_tensor, bottleneck_tensor):
  """Retrieves bottleneck values for training images, after distortions.

  If we're training with distortions like crops, scales, or flips, we have to
  recalculate the full model for every image, and so we can't use cached
  bottleneck values. Instead we find random images for the requested category,
  run them through the distortion graph, and then the full graph to get the
  bottleneck results for each.

  Args:
    sess: Current TensorFlow Session.
    image_lists: Dictionary of training images for each label.
    how_many: The integer number of bottleneck values to return.
    category: Name string of which set of images to fetch - training, testing,
    or validation.
    image_dir: Root folder string of the subfolders containing the training
    images.
    input_jpeg_tensor: The input layer we feed the image data to.
    distorted_image: The output node of the distortion graph.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: The bottleneck output layer of the CNN graph.

  Returns:
    List of bottleneck arrays and their corresponding ground truths.
  """
  class_count = len(image_lists.keys())
  bottlenecks = []
  ground_truths = []
  for unused_i in range(how_many):
    label_index = random.randrange(class_count)
    label_name = list(image_lists.keys())[label_index]
    image_index = random.randrange(MAX_NUM_IMAGES_PER_CLASS + 1)
    image_path = get_image_path(image_lists, label_name, image_index, image_dir,
                                category)
    if not gfile.Exists(image_path):
      tf.logging.fatal('File does not exist %s', image_path)
    jpeg_data = gfile.FastGFile(image_path, 'rb').read()
    # Note that we materialize the distorted_image_data as a numpy array before
    # sending running inference on the image. This involves 2 memory copies and
    # might be optimized in other implementations.
    distorted_image_data = sess.run(distorted_image,
                                    {input_jpeg_tensor: jpeg_data})
    bottleneck_values = sess.run(bottleneck_tensor,
                                 {resized_input_tensor: distorted_image_data})
    bottleneck_values = np.squeeze(bottleneck_values)
    ground_truth = np.zeros(class_count, dtype=np.float32)
    ground_truth[label_index] = 1.0
    bottlenecks.append(bottleneck_values)
    ground_truths.append(ground_truth)
  return bottlenecks, ground_truths


def should_distort_images(flip_left_right, random_crop, random_scale,
                          random_brightness):
  """Whether any distortions are enabled, from the input flags.

  Args:
    flip_left_right: Boolean whether to randomly mirror images horizontally.
    random_crop: Integer percentage setting the total margin used around the
    crop box.
    random_scale: Integer percentage of how much to vary the scale by.
    random_brightness: Integer range to randomly multiply the pixel values by.

  Returns:
    Boolean value indicating whether any distortions should be applied.
  """
  return (flip_left_right or (random_crop != 0) or (random_scale != 0) or
          (random_brightness != 0))

# Hàm augment ảnh ngẫu nhiên (crop, scale, flip, brightness)
def add_input_distortions(flip_left_right, random_crop, random_scale,
                          random_brightness, input_width, input_height,
                          input_depth, input_mean, input_std):
    """Tạo ra các hoạt động để áp dụng các biến dạng (distortions) được chỉ định.

  Trong quá trình huấn luyện, việc áp dụng các biến dạng đơn giản cho ảnh như
  cắt (crops), thay đổi tỷ lệ (scales), và lật (flips) có thể giúp cải thiện kết quả.
  Những biến dạng này phản ánh các loại biến thể mà chúng ta mong đợi trong thế giới
  thực, và do đó có thể giúp huấn luyện mô hình đối phó với dữ liệu tự nhiên
  hiệu quả hơn. Ở đây, chúng ta lấy các tham số được cung cấp và xây dựng một
  mạng lưới các hoạt động để áp dụng chúng lên một hình ảnh.

  Cắt ảnh (Cropping)
  ~~~~~~~~

  Việc cắt ảnh được thực hiện bằng cách đặt một hộp giới hạn (bounding box) tại một
  vị trí ngẫu nhiên trong ảnh gốc. Tham số cắt ảnh kiểm soát kích thước của hộp đó
  so với ảnh đầu vào. Nếu nó bằng 0, thì hộp có cùng kích thước với ảnh đầu vào và
  không có việc cắt nào được thực hiện. Nếu giá trị là 50%, thì hộp cắt sẽ có chiều
  rộng và chiều cao bằng một nửa so với ảnh đầu vào. Sơ đồ minh họa như sau:

  <      width        >
  +---------------------+
  |                     |
  |   width - crop%     |
  |   <       >         |
  |   +------+          |
  |   |      |          |
  |   |      |          |
  |   |      |          |
  |   +------+          |
  |                     |
  |                     |
  +---------------------+

  Thay đổi tỷ lệ (Scaling)
  ~~~~~~~~~~~~~

  Thay đổi tỷ lệ rất giống với cắt ảnh, ngoại trừ việc hộp giới hạn luôn
  luôn được căn giữa và kích thước của nó thay đổi ngẫu nhiên trong phạm vi
  cho trước. Ví dụ, nếu tỷ lệ phần trăm là 0, thì hộp giới hạn có cùng kích
  thước với ảnh đầu vào và không có việc thay đổi tỷ lệ nào được áp dụng. Nếu là
  50%, thì hộp giới hạn sẽ có kích thước ngẫu nhiên trong khoảng từ một nửa chiều
  rộng và chiều cao cho đến kích thước đầy đủ.

  Tham số (Args):
      flip_left_right: Kiểu Boolean, cho biết có lật ngang ảnh một cách ngẫu nhiên hay không.
      random_crop: Số nguyên phần trăm, thiết lập tổng lề được sử dụng xung quanh
      hộp cắt.
      random_scale: Số nguyên phần trăm, cho biết mức độ thay đổi tỷ lệ.
      random_brightness: Phạm vi số nguyên để nhân ngẫu nhiên các giá trị pixel.
      graph: Đồ thị.
      input_width: Kích thước chiều ngang của ảnh đầu vào dự kiến cho mô hình.
      input_height: Kích thước chiều dọc của ảnh đầu vào dự kiến cho mô hình.
      input_depth: Số kênh màu mà ảnh đầu vào dự kiến nên có.
      input_mean: Giá trị pixel nên bằng 0 trong ảnh cho đồ thị.
      input_std: Giá trị để chia các giá trị pixel trước khi nhận dạng.

  Trả về (Returns):
      Lớp đầu vào jpeg và tensor kết quả đã bị biến dạng.
  """

  jpeg_data = tf.placeholder(tf.string, name='DistortJPGInput')
  decoded_image = tf.image.decode_jpeg(jpeg_data, channels=input_depth)
  decoded_image_as_float = tf.cast(decoded_image, dtype=tf.float32)
  decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0)
  margin_scale = 1.0 + (random_crop / 100.0)
  resize_scale = 1.0 + (random_scale / 100.0)
  margin_scale_value = tf.constant(margin_scale)
  resize_scale_value = tf.random_uniform(tensor_shape.scalar(),
                                         minval=1.0,
                                         maxval=resize_scale)
  scale_value = tf.multiply(margin_scale_value, resize_scale_value)
  precrop_width = tf.multiply(scale_value, input_width)
  precrop_height = tf.multiply(scale_value, input_height)
  precrop_shape = tf.stack([precrop_height, precrop_width])
  precrop_shape_as_int = tf.cast(precrop_shape, dtype=tf.int32)
  precropped_image = tf.image.resize_bilinear(decoded_image_4d,
                                              precrop_shape_as_int)
  precropped_image_3d = tf.squeeze(precropped_image, squeeze_dims=[0])
  cropped_image = tf.random_crop(precropped_image_3d,
                                 [input_height, input_width, input_depth])
  if flip_left_right:
    flipped_image = tf.image.random_flip_left_right(cropped_image)
  else:
    flipped_image = cropped_image
  brightness_min = 1.0 - (random_brightness / 100.0)
  brightness_max = 1.0 + (random_brightness / 100.0)
  brightness_value = tf.random_uniform(tensor_shape.scalar(),
                                       minval=brightness_min,
                                       maxval=brightness_max)
  brightened_image = tf.multiply(flipped_image, brightness_value)
  offset_image = tf.subtract(brightened_image, input_mean)
  mul_image = tf.multiply(offset_image, 1.0 / input_std)
  distort_result = tf.expand_dims(mul_image, 0, name='DistortResult')
  return jpeg_data, distort_result


def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

#  Tạo lớp softmax và hàm mất mát
def add_final_training_ops(class_count, final_tensor_name, bottleneck_tensor,
                           bottleneck_tensor_size):
  """Adds a new softmax and fully-connected layer for training.

  We need to retrain the top layer to identify our new classes, so this function
  adds the right operations to the graph, along with some variables to hold the
  weights, and then sets up all the gradients for the backward pass.

  The set up for the softmax and fully-connected layers is based on:
  https://www.tensorflow.org/versions/master/tutorials/mnist/beginners/index.html

  Args:
    class_count: Integer of how many categories of things we're trying to
    recognize.
    final_tensor_name: Name string for the new final node that produces results.
    bottleneck_tensor: The output of the main CNN graph.
    bottleneck_tensor_size: How many entries in the bottleneck vector.

  Returns:
    The tensors for the training and cross entropy results, and tensors for the
    bottleneck input and ground truth input.
  """
  with tf.name_scope('input'):
    bottleneck_input = tf.placeholder_with_default(
        bottleneck_tensor,
        shape=[None, bottleneck_tensor_size],
        name='BottleneckInputPlaceholder')

    ground_truth_input = tf.placeholder(tf.float32,
                                        [None, class_count],
                                        name='GroundTruthInput')

  # Organizing the following ops as `final_training_ops` so they're easier
  # to see in TensorBoard
  layer_name = 'final_training_ops'
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      initial_value = tf.truncated_normal(
          [bottleneck_tensor_size, class_count], stddev=0.001)

      layer_weights = tf.Variable(initial_value, name='final_weights')

      variable_summaries(layer_weights)
    with tf.name_scope('biases'):
      layer_biases = tf.Variable(tf.zeros([class_count]), name='final_biases')
      variable_summaries(layer_biases)
    with tf.name_scope('Wx_plus_b'):
      logits = tf.matmul(bottleneck_input, layer_weights) + layer_biases
      tf.summary.histogram('pre_activations', logits)

  final_tensor = tf.nn.softmax(logits, name=final_tensor_name)
  tf.summary.histogram('activations', final_tensor)

  with tf.name_scope('cross_entropy'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        labels=ground_truth_input, logits=logits)
    with tf.name_scope('total'):
      cross_entropy_mean = tf.reduce_mean(cross_entropy)
  tf.summary.scalar('cross_entropy', cross_entropy_mean)

  with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(FLAGS.learning_rate)
    train_step = optimizer.minimize(cross_entropy_mean)

  return (train_step, cross_entropy_mean, bottleneck_input, ground_truth_input,
          final_tensor)


def add_evaluation_step(result_tensor, ground_truth_tensor):
  """Inserts the operations we need to evaluate the accuracy of our results.

  Args:
    result_tensor: The new final node that produces results.
    ground_truth_tensor: The node we feed ground truth data
    into.

  Returns:
    Tuple of (evaluation step, prediction).
  """
  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      prediction = tf.argmax(result_tensor, 1)
      correct_prediction = tf.equal(
          prediction, tf.argmax(ground_truth_tensor, 1))
    with tf.name_scope('accuracy'):
      evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', evaluation_step)
  return evaluation_step, prediction


def save_graph_to_file(sess, graph, graph_file_name):
  output_graph_def = graph_util.convert_variables_to_constants(
      sess, graph.as_graph_def(), [FLAGS.final_tensor_name])
  with gfile.FastGFile(graph_file_name, 'wb') as f:
    f.write(output_graph_def.SerializeToString())
  return


def prepare_file_system():
  # Setup the directory we'll write summaries to for TensorBoard
  if tf.gfile.Exists(FLAGS.summaries_dir):
    tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
  tf.gfile.MakeDirs(FLAGS.summaries_dir)
  if FLAGS.intermediate_store_frequency > 0:
    ensure_dir_exists(FLAGS.intermediate_output_graphs_dir)
  return


def create_model_info(architecture):
  """Given the name of a model architecture, returns information about it.

  There are different base image recognition pretrained models that can be
  retrained using transfer learning, and this function translates from the name
  of a model to the attributes that are needed to download and train with it.

  Args:
    architecture: Name of a model architecture.

  Returns:
    Dictionary of information about the model, or None if the name isn't
    recognized

  Raises:
    ValueError: If architecture name is unknown.
  """
  architecture = architecture.lower()
  if architecture == 'inception_v3':
    # pylint: disable=line-too-long
    data_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
    # pylint: enable=line-too-long
    bottleneck_tensor_name = 'pool_3/_reshape:0'
    bottleneck_tensor_size = 2048
    input_width = 299
    input_height = 299
    input_depth = 3
    resized_input_tensor_name = 'Mul:0'
    model_file_name = 'classify_image_graph_def.pb'
    input_mean = 128
    input_std = 128
  elif architecture.startswith('mobilenet_'):
    parts = architecture.split('_')
    if len(parts) != 3 and len(parts) != 4:
      tf.logging.error("Couldn't understand architecture name '%s'",
                       architecture)
      return None
    version_string = parts[1]
    if (version_string != '1.0' and version_string != '0.75' and
        version_string != '0.50' and version_string != '0.25'):
      tf.logging.error(
          """"The Mobilenet version should be '1.0', '0.75', '0.50', or '0.25',
  but found '%s' for architecture '%s'""",
          version_string, architecture)
      return None
    size_string = parts[2]
    if (size_string != '224' and size_string != '192' and
        size_string != '160' and size_string != '128'):
      tf.logging.error(
          """The Mobilenet input size should be '224', '192', '160', or '128',
 but found '%s' for architecture '%s'""",
          size_string, architecture)
      return None
    if len(parts) == 3:
      is_quantized = False
    else:
      if parts[3] != 'quantized':
        tf.logging.error(
            "Couldn't understand architecture suffix '%s' for '%s'", parts[3],
            architecture)
        return None
      is_quantized = True
    data_url = 'http://download.tensorflow.org/models/mobilenet_v1_'
    data_url += version_string + '_' + size_string + '_frozen.tgz'
    bottleneck_tensor_name = 'MobilenetV1/Predictions/Reshape:0'
    bottleneck_tensor_size = 1001
    input_width = int(size_string)
    input_height = int(size_string)
    input_depth = 3
    resized_input_tensor_name = 'input:0'
    if is_quantized:
      model_base_name = 'quantized_graph.pb'
    else:
      model_base_name = 'frozen_graph.pb'
    model_dir_name = 'mobilenet_v1_' + version_string + '_' + size_string
    model_file_name = os.path.join(model_dir_name, model_base_name)
    input_mean = 127.5
    input_std = 127.5
  else:
    tf.logging.error("Couldn't understand architecture name '%s'", architecture)
    raise ValueError('Unknown architecture', architecture)

  return {
      'data_url': data_url,
      'bottleneck_tensor_name': bottleneck_tensor_name,
      'bottleneck_tensor_size': bottleneck_tensor_size,
      'input_width': input_width,
      'input_height': input_height,
      'input_depth': input_depth,
      'resized_input_tensor_name': resized_input_tensor_name,
      'model_file_name': model_file_name,
      'input_mean': input_mean,
      'input_std': input_std,
  }


def add_jpeg_decoding(input_width, input_height, input_depth, input_mean,
                      input_std):
  """Adds operations that perform JPEG decoding and resizing to the graph..

  Args:
    input_width: Desired width of the image fed into the recognizer graph.
    input_height: Desired width of the image fed into the recognizer graph.
    input_depth: Desired channels of the image fed into the recognizer graph.
    input_mean: Pixel value that should be zero in the image for the graph.
    input_std: How much to divide the pixel values by before recognition.

  Returns:
    Tensors for the node to feed JPEG data into, and the output of the
      preprocessing steps.
  """
  jpeg_data = tf.placeholder(tf.string, name='DecodeJPGInput')
  decoded_image = tf.image.decode_jpeg(jpeg_data, channels=input_depth)
  decoded_image_as_float = tf.cast(decoded_image, dtype=tf.float32)
  decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0)
  resize_shape = tf.stack([input_height, input_width])
  resize_shape_as_int = tf.cast(resize_shape, dtype=tf.int32)
  resized_image = tf.image.resize_bilinear(decoded_image_4d,
                                           resize_shape_as_int)
  offset_image = tf.subtract(resized_image, input_mean)
  mul_image = tf.multiply(offset_image, 1.0 / input_std)
  return jpeg_data, mul_image


def main(_):
    tf.logging.set_verbosity(tf.logging.INFO)
    # Tạo các thư mục cần thiết để lưu trữ log, kết quả tạm thời, TensorBoard...
    prepare_file_system()

    # Lấy thông tin cấu hình của mô hình
    model_info = create_model_info(FLAGS.architecture)
    if not model_info:
        tf.logging.error('Did not recognize architecture flag')
        return -1

    # Nếu chưa có mô hình pre-trained, tự động tải về và giải nén
    maybe_download_and_extract(model_info['data_url'])

    # Load đồ thị (graph) của mô hình pre-trained, lấy tensor bottleneck và tensor đầu vào ảnh đã resize
    graph, bottleneck_tensor, resized_image_tensor = create_model_graph(model_info)

    # Phân tích thư mục dữ liệu, chia ra các tập train, test, validation
    image_lists = create_image_lists(FLAGS.image_dir, FLAGS.testing_percentage, FLAGS.validation_percentage)
    class_count = len(image_lists.keys())

    if class_count == 0:
        tf.logging.error('No valid folders of images found at ' + FLAGS.image_dir)
        return -1
    if class_count == 1:
        tf.logging.error('Only one valid folder of images found at ' + FLAGS.image_dir + ' - cần ít nhất 2 lớp.')
        return -1

    # Kiểm tra xem có áp dụng các biến đổi dữ liệu hay không (lật ảnh, crop, scale, độ sáng)
    do_distort_images = should_distort_images(
        FLAGS.flip_left_right, FLAGS.random_crop, FLAGS.random_scale, FLAGS.random_brightness)

    with tf.Session(graph=graph) as sess:

        # Tạo các tensor để đọc và tiền xử lý ảnh JPEG
        jpeg_data_tensor, decoded_image_tensor = add_jpeg_decoding(
            model_info['input_width'], model_info['input_height'],
            model_info['input_depth'], model_info['input_mean'],
            model_info['input_std'])

        if do_distort_images:
            # Nếu có áp dụng biến đổi dữ liệu, tạo pipeline biến đổi
            distorted_jpeg_data_tensor, distorted_image_tensor = add_input_distortions(
                FLAGS.flip_left_right, FLAGS.random_crop, FLAGS.random_scale,
                FLAGS.random_brightness, model_info['input_width'],
                model_info['input_height'], model_info['input_depth'],
                model_info['input_mean'], model_info['input_std'])
        else:
            # Nếu không áp dụng biến đổi, cache sẵn các giá trị bottleneck cho toàn bộ dữ liệu
            cache_bottlenecks(sess, image_lists, FLAGS.image_dir,
                              FLAGS.bottleneck_dir, jpeg_data_tensor,
                              decoded_image_tensor, resized_image_tensor,
                              bottleneck_tensor, FLAGS.architecture)

        # Thêm layer Softmax mới vào cuối mô hình, layer này sẽ được huấn luyện lại
        train_step, cross_entropy, bottleneck_input, ground_truth_input, final_tensor = add_final_training_ops(
            len(image_lists.keys()), FLAGS.final_tensor_name, bottleneck_tensor, model_info['bottleneck_tensor_size'])

        # Thêm phép đo accuracy để đánh giá mô hình
        evaluation_step, prediction = add_evaluation_step(final_tensor, ground_truth_input)

        # Gộp toàn bộ các phép đo summary để hiển thị lên TensorBoard
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train', sess.graph)
        validation_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/validation')

        # Khởi tạo toàn bộ biến số trong graph
        init = tf.global_variables_initializer()
        sess.run(init)

        # Vòng lặp huấn luyện chính
        for i in range(FLAGS.how_many_training_steps):

            if do_distort_images:
                # Lấy ngẫu nhiên batch dữ liệu đã áp dụng biến đổi
                train_bottlenecks, train_ground_truth = get_random_distorted_bottlenecks(
                    sess, image_lists, FLAGS.train_batch_size, 'training',
                    FLAGS.image_dir, distorted_jpeg_data_tensor,
                    distorted_image_tensor, resized_image_tensor, bottleneck_tensor)
            else:
                # Lấy batch dữ liệu từ cache bottleneck
                train_bottlenecks, train_ground_truth, _ = get_random_cached_bottlenecks(
                    sess, image_lists, FLAGS.train_batch_size, 'training',
                    FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
                    decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
                    FLAGS.architecture)

            # Thực hiện 1 bước train và ghi log ra TensorBoard
            train_summary, _ = sess.run([merged, train_step],feed_dict={bottleneck_input: train_bottlenecks,ground_truth_input: train_ground_truth})
            train_writer.add_summary(train_summary, i)

            # Định kỳ in ra kết quả Accuracy và Cross-Entropy
            is_last_step = (i + 1 == FLAGS.how_many_training_steps)
            if (i % FLAGS.eval_step_interval) == 0 or is_last_step:

                train_accuracy, cross_entropy_value = sess.run(
                    [evaluation_step, cross_entropy],
                    feed_dict={bottleneck_input: train_bottlenecks,
                               ground_truth_input: train_ground_truth})

                tf.logging.info('%s: Step %d: Train accuracy = %.1f%%' % (datetime.now(), i, train_accuracy * 100))
                tf.logging.info('%s: Step %d: Cross entropy = %f' % (datetime.now(), i, cross_entropy_value))

                # Đánh giá trên tập validation
                validation_bottlenecks, validation_ground_truth, _ = get_random_cached_bottlenecks(
                    sess, image_lists, FLAGS.validation_batch_size, 'validation',
                    FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
                    decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
                    FLAGS.architecture)

                validation_summary, validation_accuracy = sess.run(
                    [merged, evaluation_step],
                    feed_dict={bottleneck_input: validation_bottlenecks,
                               ground_truth_input: validation_ground_truth})
                validation_writer.add_summary(validation_summary, i)

                tf.logging.info('%s: Step %d: Validation accuracy = %.1f%% (N=%d)' %
                                (datetime.now(), i, validation_accuracy * 100, len(validation_bottlenecks)))

            # Định kỳ lưu kết quả mô hình tạm thời
            intermediate_frequency = FLAGS.intermediate_store_frequency
            if intermediate_frequency > 0 and (i % intermediate_frequency == 0) and i > 0:
                intermediate_file_name = FLAGS.intermediate_output_graphs_dir + 'intermediate_' + str(i) + '.pb'
                tf.logging.info('Save intermediate result to : ' + intermediate_file_name)
                save_graph_to_file(sess, graph, intermediate_file_name)

        # Sau khi huấn luyện xong, đánh giá trên tập test (chưa dùng lần nào)
        test_bottlenecks, test_ground_truth, test_filenames = get_random_cached_bottlenecks(
            sess, image_lists, FLAGS.test_batch_size, 'testing',
            FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
            decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
            FLAGS.architecture)

        test_accuracy, predictions = sess.run(
            [evaluation_step, prediction],
            feed_dict={bottleneck_input: test_bottlenecks, ground_truth_input: test_ground_truth})

        tf.logging.info('Final test accuracy = %.1f%% (N=%d)' % (test_accuracy * 100, len(test_bottlenecks)))

        # In ra danh sách các ảnh bị mô hình đoán sai nếu có yêu cầu
        if FLAGS.print_misclassified_test_images:
            tf.logging.info('=== MISCLASSIFIED TEST IMAGES ===')
            for i, test_filename in enumerate(test_filenames):
                if predictions[i] != test_ground_truth[i].argmax():
                    tf.logging.info('%70s  %s' % (test_filename, list(image_lists.keys())[predictions[i]]))

        # Lưu mô hình huấn luyện xong dưới dạng .pb
        save_graph_to_file(sess, graph, FLAGS.output_graph)

        # Lưu danh sách nhãn tương ứng với các lớp
        with gfile.FastGFile(FLAGS.output_labels, 'w') as f:
            f.write('\n'.join(image_lists.keys()) + '\n')

if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--image_dir',
      type=str,
      default='',
      help='Path to folders of labeled images.'
  )
  parser.add_argument(
      '--output_graph',
      type=str,
      default='/tmp/output_graph.pb',
      help='Where to save the trained graph.'
  )
  parser.add_argument(
      '--intermediate_output_graphs_dir',
      type=str,
      default='/tmp/intermediate_graph/',
      help='Where to save the intermediate graphs.'
  )
  parser.add_argument(
      '--intermediate_store_frequency',
      type=int,
      default=0,
      help="""\
         How many steps to store intermediate graph. If "0" then will not
         store.\
      """
  )
  parser.add_argument(
      '--output_labels',
      type=str,
      default='/tmp/output_labels.txt',
      help='Where to save the trained graph\'s labels.'
  )
  parser.add_argument(
      '--summaries_dir',
      type=str,
      default='/tmp/retrain_logs',
      help='Where to save summary logs for TensorBoard.'
  )
  parser.add_argument(
      '--how_many_training_steps',
      type=int,
      default=4000,
      help='How many training steps to run before ending.'
  )
  parser.add_argument(
      '--learning_rate',
      type=float,
      default=0.01,
      help='How large a learning rate to use when training.'
  )
  parser.add_argument(
      '--testing_percentage',
      type=int,
      default=10,
      help='What percentage of images to use as a test set.'
  )
  parser.add_argument(
      '--validation_percentage',
      type=int,
      default=10,
      help='What percentage of images to use as a validation set.'
  )
  parser.add_argument(
      '--eval_step_interval',
      type=int,
      default=10,
      help='How often to evaluate the training results.'
  )
  parser.add_argument(
      '--train_batch_size',
      type=int,
      default=100,
      help='How many images to train on at a time.'
  )
  parser.add_argument(
      '--test_batch_size',
      type=int,
      default=-1,
      help="""\
      How many images to test on. This test set is only used once, to evaluate
      the final accuracy of the model after training completes.
      A value of -1 causes the entire test set to be used, which leads to more
      stable results across runs.\
      """
  )
  parser.add_argument(
      '--validation_batch_size',
      type=int,
      default=100,
      help="""\
      How many images to use in an evaluation batch. This validation set is
      used much more often than the test set, and is an early indicator of how
      accurate the model is during training.
      A value of -1 causes the entire validation set to be used, which leads to
      more stable results across training iterations, but may be slower on large
      training sets.\
      """
  )
  parser.add_argument(
      '--print_misclassified_test_images',
      default=False,
      help="""\
      Whether to print out a list of all misclassified test images.\
      """,
      action='store_true'
  )
  parser.add_argument(
      '--model_dir',
      type=str,
      default='/tmp/imagenet',
      help="""\
      Path to classify_image_graph_def.pb,
      imagenet_synset_to_human_label_map.txt, and
      imagenet_2012_challenge_label_map_proto.pbtxt.\
      """
  )
  parser.add_argument(
      '--bottleneck_dir',
      type=str,
      default='/tmp/bottleneck',
      help='Path to cache bottleneck layer values as files.'
  )
  parser.add_argument(
      '--final_tensor_name',
      type=str,
      default='final_result',
      help="""\
      The name of the output classification layer in the retrained graph.\
      """
  )
  parser.add_argument(
      '--flip_left_right',
      default=False,
      help="""\
      Whether to randomly flip half of the training images horizontally.\
      """,
      action='store_true'
  )
  parser.add_argument(
      '--random_crop',
      type=int,
      default=0,
      help="""\
      A percentage determining how much of a margin to randomly crop off the
      training images.\
      """
  )
  parser.add_argument(
      '--random_scale',
      type=int,
      default=0,
      help="""\
      A percentage determining how much to randomly scale up the size of the
      training images by.\
      """
  )
  parser.add_argument(
      '--random_brightness',
      type=int,
      default=0,
      help="""\
      A percentage determining how much to randomly multiply the training image
      input pixels up or down by.\
      """
  )
  parser.add_argument(
      '--architecture',
      type=str,
      default='inception_v3',
      help="""\
      Which model architecture to use. 'inception_v3' is the most accurate, but
      also the slowest. For faster or smaller models, chose a MobileNet with the
      form 'mobilenet_<parameter size>_<input_size>[_quantized]'. For example,
      'mobilenet_1.0_224' will pick a model that is 17 MB in size and takes 224
      pixel input images, while 'mobilenet_0.25_128_quantized' will choose a much
      less accurate, but smaller and faster network that's 920 KB on disk and
      takes 128x128 images. See https://research.googleblog.com/2017/06/mobilenets-open-source-models-for.html
      for more information on Mobilenet.\
      """)
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

## evaluate.py
- Tải mô hình từ file .pb: Lấy các tensor cần thiết: đầu vào ảnh, đầu ra kết quả, logits.

- Tạo placeholder cho nhãn đúng (one-hot vector): Nhãn đúng sẽ được truyền vào khi kiểm thử từng ảnh.

- Tạo phép đo Accuracy và Cross Entropy
  - Accuracy: So sánh dự đoán với nhãn đúng.
  - Cross Entropy: Đánh giá mức độ sai lệch dự đoán.

- Chuẩn bị tập kiểm thử:
  - Đọc danh sách ảnh từ thư mục dữ liệu.
  - Gán nhãn one-hot cho từng ảnh.

- Duyệt từng ảnh kiểm thử
  - Resize ảnh, chuẩn hóa giá trị.
  - Truyền ảnh và nhãn vào mô hình.
  - Tính Accuracy và Cross Entropy từng ảnh.

- Tính kết quả trung bình: Trả về Accuracy và Cross Entropy trung bình trên toàn bộ tập kiểm thử.

**One-hot vector** là mảng chứa toàn số 0, chỉ có đúng 1 vị trí là 1, thể hiện nhãn của mẫu dữ liệu.
```
DR có 5 cấp độ bệnh: 0, 1, 2, 3, 4
One-hot vector của cấp 2 là [0, 0, 1, 0, 0]
```
**Placeholder cho ground truth**  là nơi truyền nhãn đúng (one-hot vector) vào mô hình khi chạy session.
```
[None, 5]:
None - số lượng mẫu ảnh có thể thay đổi tùy batch
5 - số lớp, tương ứng 5 phần tử trong one-hot vector
```
**Logits** là đầu ra thô của lớp cuối cùng của mô hình, trước khi áp dụng hàm softmax, chưa phải xác suất, nhưng cần thiết để tính toán hàm mất mát Cross-Entropy.








In [ ]:
import scripts.retrain as retrain
from scripts.count_ops import load_graph
import tensorflow as tf
import numpy as np
import os, sys
from PIL import Image

def evaluate_graph(graph_file_name):
        with load_graph(graph_file_name).as_default() as graph:
        # Placeholder cho nhãn ground truth (one-hot vector), ở đây 5 lớp <=> 5 cấp độ bệnh
        ground_truth_input = tf.placeholder(tf.float32, [None, 5], name='GroundTruthInput')

        # Lấy tensor đầu vào ảnh và đầu ra kết quả từ graph
        image_buffer_input = graph.get_tensor_by_name('input:0')
        final_tensor = graph.get_tensor_by_name('final_result:0')

        # Tạo phép đo Accuracy so sánh đầu ra với nhãn đúng
        accuracy, _ = retrain.add_evaluation_step(final_tensor, ground_truth_input)

        # Lấy tensor logits trước khi qua softmax để tính Cross Entropy
        logits = graph.get_tensor_by_name("final_training_ops/Wx_plus_b/add:0")

        # Tính toán giá trị mất mát Cross Entropy cho từng mẫu
        xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=ground_truth_input,
            logits=logits))

    image_dir = 'tf_files/flower_photos'
    testing_percentage = 10  # 10%
    validation_percentage = 10  # 10%
    validation_batch_size = 100
    category = 'testing'  # Lựa chọn tập testing

    # Tạo danh sách ảnh phân chia theo lớp và tập dữ liệu
    image_lists = retrain.create_image_lists(image_dir, testing_percentage, validation_percentage)
    class_count = len(image_lists.keys())  # Đếm số lớp trong tập dữ liệu

    ground_truths = []  # Danh sách nhãn ground truth
    filenames = []  # Danh sách đường dẫn ảnh

    # Duyệt qua từng lớp và từng ảnh
    for label_index, label_name in enumerate(image_lists.keys()):
        for image_index, image_name in enumerate(image_lists[label_name][category]):
            image_name = retrain.get_image_path(image_lists, label_name, image_index, image_dir, category)

            # Tạo nhãn one-hot cho ảnh hiện tại
            ground_truth = np.zeros([1, class_count], dtype=np.float32)
            ground_truth[0, label_index] = 1.0

            ground_truths.append(ground_truth)
            filenames.append(image_name)

    accuracies = []
    xents = []  # Lưu kết quả Cross Entropy từng ảnh

    # Mở session TensorFlow để chạy kiểm thử
    with tf.Session(graph=graph) as sess:

        for filename, ground_truth in zip(filenames, ground_truths):

            # Đọc ảnh, resize về 224x224, chuẩn hóa dữ liệu về [-1, 1]
            image = Image.open(filename).resize((224, 224), Image.ANTIALIAS)
            image = np.array(image, dtype=np.float32)[None, ...]
            image = (image - 128) / 128.0

            # Tạo dict truyền vào mô hình
            feed_dict = {
                image_buffer_input: image,
                ground_truth_input: ground_truth
            }

            eval_accuracy, eval_xent = sess.run([accuracy, xent], feed_dict)
            accuracies.append(eval_accuracy)
            xents.append(eval_xent)

    # Trả về độ chính xác trung bình và Cross Entropy trung bình trên toàn bộ tập kiểm thử
    return np.mean(accuracies), np.mean(xents)

if __name__ == "__main__":
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    # Lấy tham số dòng lệnh (đường dẫn file mô hình .pb)
    accuracy, xent = evaluate_graph(*sys.argv[1:])
    print('Accuracy: %g' % accuracy)
    print('Cross Entropy: %g' % xent)


## quantize_graph.py
  - Giảm kích thước mô hình .pb sau khi huấn luyện
  - Lượng tử hóa: Chuyển các giá trị thực (float32) sang số nguyên ít bit hơn (int8 hoặc dạng lượng tử khác) nhằm giảm kích thước model từ 100MB -> vài MB: vd
    - từ 0.000123, 0.56789, -0.43212, 1.23456 - float32, trọng số ban đầu
    - thành 3, 144, 22, 255  --> uint8, lượng tử hóa 8-bit


  - Tối ưu mô hình cho các thiết bị nhúng, điện thoại, máy tính biên (Edge AI)

**Tác động:**

  - Tăng tốc độ suy luận
  - Giảm băng thông truyền tải mô hình
  - Giảm tiêu thụ tài nguyên trên thiết bị
  
`GraphDef là định dạng chuẩn của TensorFlow để biểu diễn cấu trúc của mô hình dưới dạng đồ thị tính toán.`



In [ ]:
flags = flags_lib
FLAGS = flags.FLAGS

flags.DEFINE_boolean("print_nodes", False, """Lists all nodes in the model.""")
flags.DEFINE_string("input", "", "Đường dẫn GraphDef gốc cần lượng tử hóa.")
flags.DEFINE_string("output_node_names", "","""Output node names, comma separated.""")
flags.DEFINE_string("output", "", """File to save the output graph to.""")
flags.DEFINE_integer("bitdepth", 8,"""Số bit lượng tử hóa, mặc định 8-bit.""")
flags.DEFINE_string("mode", "round","""Chế độ lượng tử hóa: round, quantize, eightbit, weights, weights_rounded""")
flags.DEFINE_string("test_input_dims", "1,224,224,3", "Kích thước tensor đầu vào khi test mô hình""")
flags.DEFINE_boolean("strip_redundant_quantization", True,"""Removes redundant dequantize/quantize pairs.""")
flags.DEFINE_boolean("quantized_input", False, "Đầu vào đã lượng tử hóa sẵn chưa. Only supported when --mode=eightbit")
flags.DEFINE_float("quantized_input_min", 0, "Giá trị min của dữ liệu đầu vào khi lượng tử hóa")
flags.DEFINE_float("quantized_input_max", 1,"Giá trị max của dữ liệu đầu vào khi lượng tử hóa")
flags.DEFINE_float("quantized_fallback_min", None, "Giá trị min mặc định nếu thiếu thông tin min-max trong đồ thị")
flags.DEFINE_float("quantized_fallback_max", None, "Giá trị max mặc định nếu thiếu thông tin min-max trong đồ thị")


def print_input_nodes(current_node, nodes_map, indent, already_visited):
    # In ra tên và loại toán tử của node hiện tại, có thụt lề theo cấp
    print(" " * indent + current_node.op + ":" + current_node.name)
    already_visited[current_node.name] = True  # Đánh dấu đã thăm node này
    # Duyệt qua tất cả các node đầu vào của node hiện tại
    for input_node_name in current_node.input:
        if input_node_name in already_visited:
            continue  # Nếu đã duyệt thì bỏ qua tránh lặp vô hạn
        input_node = nodes_map[input_node_name]  # Lấy thông tin node từ map
        print_input_nodes(input_node, nodes_map, indent + 1, already_visited)  # Đệ quy in tiếp

def create_node(op, name, inputs):
    # Tạo một node mới với loại toán tử op và danh sách node đầu vào inputs
    new_node = node_def_pb2.NodeDef()
    new_node.op = op
    new_node.name = name
    for input_name in inputs:
        new_node.input.extend([input_name])  # Thêm các node đầu vào
    return new_node

def create_constant_node(name, value, dtype, shape=None):
    node = create_node("Const", name, [])
    set_attr_dtype(node, "dtype", dtype)  # Gán kiểu dữ liệu cho node
    set_attr_tensor(node, "value", value, dtype, shape)  # Gán giá trị tensor
    return node


def copy_attr(node, key, attr_value):
  try:
    node.attr[key].CopyFrom(attr_value)
  except KeyError:
    pass

def set_attr_dtype(node, key, value):
  try:
    node.attr[key].CopyFrom(attr_value_pb2.AttrValue(type=value.as_datatype_enum))
  except KeyError:
    pass

def set_attr_shape(node, key, value):
  try:
    node.attr[key].CopyFrom(attr_value_pb2.AttrValue(shape=tensor_shape.as_shape(value).as_proto()))
  except KeyError:
    pass


def set_attr_tensor(node, key, value, dtype, shape=None):
  try:
    node.attr[key].CopyFrom(
        attr_value_pb2.AttrValue(tensor=tensor_util.make_tensor_proto(
            value, dtype=dtype, shape=shape)))
  except KeyError:
    pass


def set_attr_string(node, key, value):
  try:
    node.attr[key].CopyFrom(attr_value_pb2.AttrValue(s=value))
  except KeyError:
    pass


def set_attr_int_list(node, key, value):
  list_value = attr_value_pb2.AttrValue.ListValue(i=value)
  try:
    node.attr[key].CopyFrom(attr_value_pb2.AttrValue(list=list_value))
  except KeyError:
    pass


def set_attr_bool(node, key, value):
  try:
    node.attr[key].CopyFrom(attr_value_pb2.AttrValue(b=value))
  except KeyError:
    pass


def set_attr_int(node, key, value):
  try:
    node.attr[key].CopyFrom(attr_value_pb2.AttrValue(i=value))
  except KeyError:
    pass


def set_attr_float(node, key, value):
  try:
    node.attr[key].CopyFrom(attr_value_pb2.AttrValue(f=value))
  except KeyError:
    pass

def node_name_from_input(node_name):
    """Tách bỏ phần cổng (port) hoặc ký tự đặc biệt khỏi tên node."""
    if node_name.startswith("^"):
        node_name = node_name[1:]  # Bỏ dấu ^ nếu có
    m = re.search(r"(.*):\d+$", node_name)
    if m:
        node_name = m.group(1)  # Chỉ lấy phần tên trước dấu :
    return node_name

def ensure_tensor_name_has_port(node_name):
    """Đảm bảo tên tensor có phần chỉ định cổng :0 nếu chưa có."""
    m = re.search(r"(.*):\d+$", node_name)
    if m:
        return node_name
    else:
        return node_name + ":0"

def unique_node_name_from_input(node_name):
    """Chuyển tên node thành dạng hợp lệ để tạo node mới."""
    return node_name.replace(":", "__port__").replace("^", "__hat__")

def quantize_array(arr, num_buckets):
    """Lượng tử hóa mảng numpy thành các khoảng (bucket).
    Ví dụ:
    arr = [0, 0.3, 0.6, 1], num_buckets = 2
    Kết quả: [0.25, 0.25, 0.75, 0.75]
    """
    if num_buckets < 1:
        raise ValueError("num_buckets phải >= 1")

    arr_max = arr.max()
    arr_min = arr.min()

    if arr_max == arr_min:
        return arr  # Nếu toàn bộ giá trị giống nhau thì không lượng tử hóa

    bucket_width = (arr_max - arr_min) / num_buckets  # Độ rộng mỗi khoảng

    # Xác định bucket của từng phần tử
    bucket_indices = np.floor((arr - arr_min) / bucket_width)
    bucket_indices[bucket_indices == num_buckets] = num_buckets - 1

    # Gán giá trị là trung tâm của mỗi khoảng
    arr = arr_min + bucket_width * (bucket_indices + 0.5)
    return arr

def quantize_weight_rounded(input_node):
    """Tạo node trọng số mới với giá trị đã lượng tử hóa dạng làm tròn."""
    input_tensor = input_node.attr["value"].tensor
    tensor_value = tensor_util.MakeNdarray(input_tensor)
    shape = input_tensor.tensor_shape

    num_buckets = 1 << FLAGS.bitdepth  # Số lượng bucket, thường là 2^bitdepth

    tensor_value_rounded = quantize_array(tensor_value, num_buckets)
    tensor_shape_list = tensor_util.TensorShapeProtoToList(shape)

    return [
        create_constant_node(input_node.name, tensor_value_rounded, dtypes.float32, shape=tensor_shape_list)
    ]

def quantize_weight_eightbit(input_node, quantization_mode):
    """Tạo các node lượng tử hóa trọng số theo chuẩn 8-bit."""
    base_name = input_node.name + "_"
    quint8_const_name = base_name + "quint8_const"
    min_name = base_name + "min"
    max_name = base_name + "max"

    float_tensor = tensor_util.MakeNdarray(input_node.attr["value"].tensor)
    min_value = np.min(float_tensor.flatten())
    max_value = np.max(float_tensor.flatten())

    # Đảm bảo khoảng giá trị bao gồm số 0
    if min_value > 0.0:
        min_value = 0.0

    # Trường hợp min == max cần xử lý đặc biệt để tránh lỗi
    if min_value == max_value:
        if abs(min_value) < 1e-6:
            max_value = min_value + 1.0
        elif min_value > 0:
            max_value = 2 * min_value
        else:
            max_value = min_value / 2.0

    # Lượng tử hóa tensor sang dạng uint8
    sess = session.Session()
    with sess.as_default():
        quantize_op = array_ops.quantize_v2(float_tensor, min_value, max_value, dtypes.quint8, mode=quantization_mode)
        quint8_tensor = quantize_op[0].eval()

    shape = tensor_util.TensorShapeProtoToList(input_node.attr["value"].tensor.tensor_shape)

    # Tạo các node: hằng số uint8, min, max, dequantize
    quint8_const_node = create_constant_node(quint8_const_name, quint8_tensor, dtypes.quint8, shape=shape)
    min_node = create_constant_node(min_name, min_value, dtypes.float32)
    max_node = create_constant_node(max_name, max_value, dtypes.float32)
    dequantize_node = create_node("Dequantize", input_node.name, [quint8_const_name, min_name, max_name])

    set_attr_dtype(dequantize_node, "T", dtypes.quint8)
    set_attr_string(dequantize_node, "mode", quantization_mode)

    return [quint8_const_node, min_node, max_node, dequantize_node]

EightbitizeRecursionState = collections.namedtuple(
    "EightbitizeRecursionState",
    ["already_visited", "output_node_stack", "merged_with_fake_quant"])

# class GraphRewriter(object) này viết lại đồ thị TensorFlow:
# Duyệt toàn bộ node theo thứ tự phụ thuộc
# Quyết định có lượng tử hóa node nào không
# Chèn node mới vào self.output_graph
# Trả về đồ thị lượng tử hóa để lưu ra file .pb
class GraphRewriter(object):
  """Takes a float graph, and rewrites it in quantized form."""

  def __init__(self,
               input_graph,
               mode,
               quantized_input_range,
               fallback_quantization_range=None):
    """Sets up the class to rewrite a float graph.

    Args:
      input_graph: A float graph to transform.
      mode: A string controlling how quantization is performed -
        round, quantize, eightbit, or weights.
      quantized_input_range: if set, assume the input is
        quantized and represents the range
        [quantized_input_range[0], quantized_input_range[1]]
      fallback_quantization_range: if set, then for nodes where the quantization
        range can't be inferred from the graph, use the range
        [fallback_quantization_range[0], fallback_quantization_range[1]) instead
        of using a RequantizationRange node in the graph.

    Raises:
      ValueError: Two nodes with the same name were found in the graph.
    """
    self.input_graph = input_graph
    self.nodes_map = self.create_nodes_map(input_graph)
    self.output_graph = None
    self.mode = mode
    self.final_node_renames = {}
    if quantized_input_range:
      self.input_range = (quantized_input_range[0], quantized_input_range[1])
      if self.input_range[0] >= self.input_range[1]:
        raise ValueError("Invalid quantized_input_range: [%s,%s]" %
                         self.input_range)
      if self.mode != "eightbit":
        raise ValueError(
            "quantized_input_range can only be specified in eightbit mode")
    else:
      self.input_range = None

    if fallback_quantization_range:
      self.fallback_quantization_range = [
          fallback_quantization_range[0], fallback_quantization_range[1]
      ]
      if (self.fallback_quantization_range[0] >=
          self.fallback_quantization_range[1]):
        raise ValueError("Invalid fallback_quantization_range: [%s,%s]" %
                         self.fallback_quantization_range)
      if self.mode != "eightbit":
        raise ValueError("fallback_quantization_range can only be "
                         "specified in eightbit mode")
    else:
      self.fallback_quantization_range = None

    # Data that is valid only during the recursive call to rewrite the graph.
    self.state = None

  def create_nodes_map(self, graph):
    """Builds a mapping of node names to their defs from the graph."""
    nodes_map = {}
    for node in graph.node:
      if node.name not in nodes_map.keys():
        nodes_map[node.name] = node
      else:
        raise ValueError("Duplicate node names detected.")
    return nodes_map

  def rewrite(self, output_node_names):
    """Triggers rewriting of the float graph.

    Args:
      output_node_names: A list of names of the nodes that produce the final
        results.

    Returns:
      A quantized version of the float graph.
    """
    self.output_graph = graph_pb2.GraphDef()
    output_nodes = [
        self.nodes_map[output_node_name]
        for output_node_name in output_node_names
    ]
    if self.mode == "round":
      self.already_visited = {}
      for output_node in output_nodes:
        self.round_nodes_recursively(output_node)
    elif self.mode == "quantize":
      self.already_visited = {}
      self.already_quantized = {}
      for output_node in output_nodes:
        self.quantize_nodes_recursively(output_node)
    elif self.mode == "eightbit":
      self.set_input_graph(graph_util.remove_training_nodes(self.input_graph))
      output_nodes = [
          self.nodes_map[output_node_name]
          for output_node_name in output_node_names
      ]

      self.state = EightbitizeRecursionState(
          already_visited={}, output_node_stack=[], merged_with_fake_quant={})
      for output_node in output_nodes:
        self.eightbitize_nodes_recursively(output_node)
      self.state = None
      if self.input_range:
        self.add_output_graph_node(
            create_constant_node("quantized_input_min_value", self.input_range[
                0], dtypes.float32, []))
        self.add_output_graph_node(
            create_constant_node("quantized_input_max_value", self.input_range[
                1], dtypes.float32, []))
      if self.fallback_quantization_range:
        self.add_output_graph_node(
            create_constant_node("fallback_quantization_min_value",
                                 self.fallback_quantization_range[0],
                                 dtypes.float32, []))
        self.add_output_graph_node(
            create_constant_node("fallback_quantization_max_value",
                                 self.fallback_quantization_range[1],
                                 dtypes.float32, []))
      if FLAGS.strip_redundant_quantization:
        self.output_graph = self.remove_redundant_quantization(
            self.output_graph)
        self.remove_dead_nodes(output_node_names)
      self.apply_final_node_renames()
    elif self.mode == "weights":
      self.output_graph = self.quantize_weights(self.input_graph,
                                                b"MIN_COMBINED")
      self.remove_dead_nodes(output_node_names)
    elif self.mode == "weights_rounded":
      self.output_graph = self.quantize_weights(self.input_graph, self.mode)
      self.remove_dead_nodes(output_node_names)
    else:
      print("Bad mode - " + self.mode + ".")
    return self.output_graph

  def round_nodes_recursively(self, current_node):
    """The entry point for simple rounding quantization."""
    if self.already_visited[current_node.name]:
      return
    self.already_visited[current_node.name] = True
    for input_node_name in current_node.input:
      input_node_name = node_name_from_input(input_node_name)
      input_node = self.nodes_map[input_node_name]
      self.round_nodes_recursively(input_node)
    nodes_to_quantize = ["Conv2D", "BiasAdd", "MatMul"]
    if any(current_node.op in s for s in nodes_to_quantize):
      new_node = node_def_pb2.NodeDef()
      new_node.CopyFrom(current_node)
      new_node.name = current_node.name + "_original"
      self.add_output_graph_node(new_node)
      levels = 1 << FLAGS.bitdepth
      constant_name = current_node.name + "_round_depth"
      constant_tensor = constant_op.constant(
          levels, dtype=dtypes.int32, name=constant_name)
      constant_node = constant_tensor.op.node_def
      self.add_output_graph_node(constant_node)
      quantize_node = node_def_pb2.NodeDef()
      quantize_node.op = "RoundToSteps"
      quantize_node.name = current_node.name
      quantize_node.input.extend([current_node.name + "_original"])
      quantize_node.input.extend([constant_node.name])
      self.add_output_graph_node(quantize_node)
    else:
      new_node = node_def_pb2.NodeDef()
      new_node.CopyFrom(current_node)
      self.add_output_graph_node(new_node)

  def quantize_nodes_recursively(self, current_node):
    """The entry point for quantizing nodes to eight bit and back."""
    if self.already_visited[current_node.name]:
      return
    self.already_visited[current_node.name] = True
    for input_node_name in current_node.input:
      input_node_name = node_name_from_input(input_node_name)
      input_node = self.nodes_map[input_node_name]
      self.quantize_nodes_recursively(input_node)
    nodes_to_quantize = ["Conv2D", "BiasAdd", "MatMul"]
    if any(current_node.op in s for s in nodes_to_quantize):
      for input_name in current_node.input:
        input_name = node_name_from_input(input_name)
        input_node = self.nodes_map[input_name]
        self.quantize_node(input_node)
      self.quantize_node(current_node)
    else:
      new_node = node_def_pb2.NodeDef()
      new_node.CopyFrom(current_node)
      self.add_output_graph_node(new_node)

  def quantize_node(self, input_node):
    """Handles quantizing a single node."""
    input_name = input_node.name
    if input_name in self.already_quantized:
      return
    self.already_quantized[input_name] = True
    original_input_name = input_name + "_original"
    reshape_name = input_name + "_reshape"
    reshape_dims_name = input_name + "_reshape_dims"
    max_name = input_name + "_max"
    min_name = input_name + "_min"
    dims_name = input_name + "_dims"
    quantize_name = input_name + "_quantize"
    dequantize_name = input_name
    original_input_node = node_def_pb2.NodeDef()
    original_input_node.CopyFrom(input_node)
    original_input_node.name = original_input_name
    self.add_output_graph_node(original_input_node)
    reshape_dims_node = create_constant_node(reshape_dims_name, -1,
                                             dtypes.int32, [1])
    self.add_output_graph_node(reshape_dims_node)
    reshape_node = create_node("Reshape", reshape_name,
                               [original_input_name, reshape_dims_name])
    set_attr_dtype(reshape_node, "T", dtypes.float32)
    self.add_output_graph_node(reshape_node)
    dims_node = create_constant_node(dims_name, 0, dtypes.int32, [1])
    self.add_output_graph_node(dims_node)
    max_node = create_node("Max", max_name, [reshape_name, dims_name])
    set_attr_dtype(max_node, "T", dtypes.float32)
    set_attr_bool(max_node, "keep_dims", False)
    self.add_output_graph_node(max_node)
    min_node = create_node("Min", min_name, [reshape_name, dims_name])
    set_attr_dtype(min_node, "T", dtypes.float32)
    set_attr_bool(min_node, "keep_dims", False)
    self.add_output_graph_node(min_node)
    quantize_node = create_node("Quantize", quantize_name,
                                [original_input_name, min_name, max_name])
    set_attr_dtype(quantize_node, "T", dtypes.quint8)
    set_attr_string(quantize_node, "mode", b"MIN_FIRST")
    self.add_output_graph_node(quantize_node)
    dequantize_node = create_node("Dequantize", dequantize_name,
                                  [quantize_name, min_name, max_name])
    set_attr_dtype(dequantize_node, "T", dtypes.quint8)
    set_attr_string(dequantize_node, "mode", b"MIN_FIRST")
    self.add_output_graph_node(dequantize_node)

  def should_merge_with_fake_quant_node(self):
    """Should the current node merge with self.state.output_node_stack[-1]?"""
    if not self.state.output_node_stack:
      return False
    top = self.state.output_node_stack[-1]
    return top[1] == 0 and top[0].op in ["FakeQuantWithMinMaxVars"]

  def should_quantize_const(self, node):
    if not self.state.output_node_stack:
      return False
    top = self.state.output_node_stack[-1]
    if not top[2]:
      return False
    dtype = dtypes.as_dtype(node.attr["dtype"].type)
    assert dtype == dtypes.float32, (
        "Failed to quantized constant %s of type %s" % (node.name, dtype))
    return True

  def eightbitize_nodes_recursively(self, current_node):
    """The entry point for transforming a graph into full eight bit."""
    if current_node.name in self.state.already_visited:
      if (self.should_merge_with_fake_quant_node() or
          current_node.name in self.state.merged_with_fake_quant):
        raise ValueError("Unsupported graph structure: output of node %s "
                         "is processed by a FakeQuant* node and should have "
                         "no other outputs.", current_node.name)
      return
    self.state.already_visited[current_node.name] = True

    for i, input_node_name in enumerate(current_node.input):
      quantize_input = False
      if current_node.op in ("MatMul", "Conv2D", "BiasAdd", "MaxPool",
                             "AvgPool", "Relu", "Relu6",
                             "BatchNormWithGlobalNormalization"):
        quantize_input = True
      elif current_node.op == "Concat" and i > 0:
        quantize_input = (
            dtypes.as_dtype(current_node.attr["T"].type) == dtypes.float32)
      elif current_node.op == "Reshape" and i == 0:
        quantize_input = (
            dtypes.as_dtype(current_node.attr["T"].type) == dtypes.float32)

      self.state.output_node_stack.append((current_node, i, quantize_input))

      input_node_name = node_name_from_input(input_node_name)
      input_node = self.nodes_map[input_node_name]
      self.eightbitize_nodes_recursively(input_node)

      self.state.output_node_stack.pop()

    if current_node.op == "MatMul":
      self.eightbitize_mat_mul_node(current_node)
    elif current_node.op == "Conv2D":
      self.eightbitize_conv_node(current_node)
    elif current_node.op == "BiasAdd":
      self.eightbitize_bias_add_node(current_node)
    elif current_node.op == "MaxPool" or current_node.op == "AvgPool":
      self.eightbitize_single_input_tensor_node(current_node,
                                                self.add_pool_function)
    elif current_node.op == "Relu" or current_node.op == "Relu6":
      self.eightbitize_single_input_tensor_node(current_node,
                                                self.add_relu_function)
    elif (current_node.op == "Concat" and
          dtypes.as_dtype(current_node.attr["T"].type) == dtypes.float32):
      self.eightbitize_concat_node(current_node)
    elif current_node.op == "BatchNormWithGlobalNormalization":
      self.eightbitize_batch_norm_node(current_node)
    elif (current_node.op == "Reshape" and
          dtypes.as_dtype(current_node.attr["T"].type) == dtypes.float32):
      self.eightbitize_reshape_node(current_node)
    elif (self.input_range and
          current_node.op in ("Placeholder", "PlaceholderV2")):
      self.eightbitize_placeholder_node(current_node)
    elif current_node.op == "FakeQuantWithMinMaxVars":
      # It will have been merged into the underlying node.
      pass
    elif current_node.op == "Const":
      if self.should_quantize_const(current_node):
        for n in quantize_weight_eightbit(current_node, b"MIN_FIRST"):
          self.add_output_graph_node(n)
      else:
        new_node = node_def_pb2.NodeDef()
        new_node.CopyFrom(current_node)
        self.add_output_graph_node(new_node)

    ###################################################################
    # Note: if more cases are added here, you may need to update the op
    # name lists in the loop over children at the start of the function.
    ###################################################################
    else:
      new_node = node_def_pb2.NodeDef()
      new_node.CopyFrom(current_node)
      self.add_output_graph_node(new_node)

    if (self.should_merge_with_fake_quant_node() and
        current_node.name not in self.state.merged_with_fake_quant):
      raise ValueError(
          "FakeQuant* node %s failed to merge with node %s of type %s" %
          (self.state.output_node_stack[-1][0], current_node.name,
           current_node.op))

  def add_eightbit_prologue_nodes(self, original_node):
    """Adds input conversion nodes to handle quantizing the underlying node."""
    namespace_prefix = original_node.name + "_eightbit"
    reshape_dims_name, reduction_dims_name = self.add_common_quantization_nodes(
        namespace_prefix)
    input_names = []
    min_max_names = []
    for original_input_name in original_node.input:
      quantize_input_name, min_input_name, max_input_name = (
          self.eightbitize_input_to_node(namespace_prefix, original_input_name,
                                         reshape_dims_name,
                                         reduction_dims_name))
      input_names.append(quantize_input_name)
      min_max_names.append(min_input_name)
      min_max_names.append(max_input_name)
    all_input_names = []
    all_input_names.extend(input_names)
    all_input_names.extend(min_max_names)
    return all_input_names

  def add_common_quantization_nodes(self, namespace_prefix):
    """Builds constant nodes needed for quantization of inputs."""
    reshape_dims_name = namespace_prefix + "_reshape_dims"
    reduction_dims_name = namespace_prefix + "_reduction_dims"

    reshape_dims_node = create_constant_node(reshape_dims_name, -1,
                                             dtypes.int32, [1])
    self.add_output_graph_node(reshape_dims_node)
    reduction_dims_node = create_constant_node(reduction_dims_name, 0,
                                               dtypes.int32, [1])
    self.add_output_graph_node(reduction_dims_node)
    return reshape_dims_name, reduction_dims_name

  def eightbitize_input_to_node(self, namespace_prefix, original_input_name,
                                reshape_dims_name, reduction_dims_name):
    """Takes one float input to an op, and converts it to quantized form."""
    unique_input_name = unique_node_name_from_input(original_input_name)
    reshape_input_name = namespace_prefix + "_reshape_" + unique_input_name
    min_input_name = namespace_prefix + "_min_" + unique_input_name
    max_input_name = namespace_prefix + "_max_" + unique_input_name
    quantize_input_name = namespace_prefix + "_quantize_" + unique_input_name
    reshape_input_node = create_node("Reshape", reshape_input_name,
                                     [original_input_name, reshape_dims_name])
    set_attr_dtype(reshape_input_node, "T", dtypes.float32)
    self.add_output_graph_node(reshape_input_node)
    min_input_node = create_node("Min", min_input_name,
                                 [reshape_input_name, reduction_dims_name])
    set_attr_dtype(min_input_node, "T", dtypes.float32)
    set_attr_bool(min_input_node, "keep_dims", False)
    self.add_output_graph_node(min_input_node)
    max_input_node = create_node("Max", max_input_name,
                                 [reshape_input_name, reduction_dims_name])
    set_attr_dtype(max_input_node, "T", dtypes.float32)
    set_attr_bool(max_input_node, "keep_dims", False)
    self.add_output_graph_node(max_input_node)
    quantize_input_node = create_node(
        "QuantizeV2", quantize_input_name,
        [original_input_name, min_input_name, max_input_name])
    set_attr_dtype(quantize_input_node, "T", dtypes.quint8)
    set_attr_string(quantize_input_node, "mode", b"MIN_FIRST")
    self.add_output_graph_node(quantize_input_node)
    min_output_name = quantize_input_name + ":1"
    max_output_name = quantize_input_name + ":2"
    return quantize_input_name, min_output_name, max_output_name

  def add_quantize_down_nodes(self, original_node, quantized_output_name):
    quantized_outputs = [
        quantized_output_name, quantized_output_name + ":1",
        quantized_output_name + ":2"
    ]
    min_max_inputs = None
    if self.should_merge_with_fake_quant_node():
      # Use the inputs to the FakeQuantWithMinMaxVars node as the inputs to
      # Requantize.
      fake_quant_node = self.state.output_node_stack[-1][0]
      min_max_inputs = [fake_quant_node.input[1], fake_quant_node.input[2]]
      assert original_node.name not in self.state.merged_with_fake_quant
      self.state.merged_with_fake_quant[original_node.name] = True
    elif self.fallback_quantization_range:
      min_max_inputs = [
          "fallback_quantization_min_value:0",
          "fallback_quantization_max_value:0"
      ]
    else:
      # Add a RequantizationRange node for finding the min and max values.
      requant_range_node = create_node(
          "RequantizationRange", original_node.name + "_eightbit_requant_range",
          quantized_outputs)
      set_attr_dtype(requant_range_node, "Tinput", dtypes.qint32)
      self.add_output_graph_node(requant_range_node)
      min_max_inputs = [
          requant_range_node.name + ":0", requant_range_node.name + ":1"
      ]
    requantize_node = create_node("Requantize",
                                  original_node.name + "_eightbit_requantize",
                                  quantized_outputs + min_max_inputs)
    set_attr_dtype(requantize_node, "Tinput", dtypes.qint32)
    set_attr_dtype(requantize_node, "out_type", dtypes.quint8)
    self.add_output_graph_node(requantize_node)
    return requantize_node.name

  def add_dequantize_result_node(self,
                                 quantized_output_name,
                                 original_node_name,
                                 min_tensor_index=1):
    min_max_inputs = [
        "%s:%s" % (quantized_output_name, min_tensor_index),
        "%s:%s" % (quantized_output_name, (min_tensor_index + 1))
    ]
    dequantize_name = original_node_name
    if self.should_merge_with_fake_quant_node():
      fake_quant_node = self.state.output_node_stack[-1][0]
      if original_node_name not in self.state.merged_with_fake_quant:
        min_max_inputs = [fake_quant_node.input[1], fake_quant_node.input[2]]
        self.state.merged_with_fake_quant[original_node_name] = True
      dequantize_name = fake_quant_node.name

    dequantize_node = create_node(
        "Dequantize", dequantize_name,
        [quantized_output_name, min_max_inputs[0], min_max_inputs[1]])
    set_attr_dtype(dequantize_node, "T", dtypes.quint8)
    set_attr_string(dequantize_node, "mode", b"MIN_FIRST")
    self.add_output_graph_node(dequantize_node)

  def eightbitize_mat_mul_node(self, original_node):
    """Replaces a MatMul node with the eight bit equivalent sub-graph."""
    quantized_mat_mul_name = original_node.name + "_eightbit_quantized_mat_mul"
    all_input_names = self.add_eightbit_prologue_nodes(original_node)
    quantized_mat_mul_node = create_node("QuantizedMatMul",
                                         quantized_mat_mul_name,
                                         all_input_names)
    set_attr_dtype(quantized_mat_mul_node, "T1", dtypes.quint8)
    set_attr_dtype(quantized_mat_mul_node, "T2", dtypes.quint8)
    set_attr_dtype(quantized_mat_mul_node, "Toutput", dtypes.qint32)
    copy_attr(quantized_mat_mul_node, "transpose_a",
              original_node.attr["transpose_a"])
    copy_attr(quantized_mat_mul_node, "transpose_b",
              original_node.attr["transpose_b"])
    self.add_output_graph_node(quantized_mat_mul_node)
    quantize_down_name = self.add_quantize_down_nodes(original_node,
                                                      quantized_mat_mul_name)
    self.add_dequantize_result_node(quantize_down_name, original_node.name)

  def eightbitize_conv_node(self, original_node):
    """Replaces a Conv2D node with the eight bit equivalent sub-graph."""
    all_input_names = self.add_eightbit_prologue_nodes(original_node)
    quantized_conv_name = original_node.name + "_eightbit_quantized_conv"
    quantized_conv_node = create_node("QuantizedConv2D", quantized_conv_name,
                                      all_input_names)
    copy_attr(quantized_conv_node, "strides", original_node.attr["strides"])
    copy_attr(quantized_conv_node, "padding", original_node.attr["padding"])
    set_attr_dtype(quantized_conv_node, "Tinput", dtypes.quint8)
    set_attr_dtype(quantized_conv_node, "Tfilter", dtypes.quint8)
    set_attr_dtype(quantized_conv_node, "out_type", dtypes.qint32)
    self.add_output_graph_node(quantized_conv_node)
    quantize_down_name = self.add_quantize_down_nodes(original_node,
                                                      quantized_conv_name)
    self.add_dequantize_result_node(quantize_down_name, original_node.name)

  def eightbitize_bias_add_node(self, original_node):
    """Replaces a BiasAdd node with the eight bit equivalent sub-graph."""
    quantized_bias_add_name = (
        original_node.name + "_eightbit_quantized_bias_add")
    all_input_names = self.add_eightbit_prologue_nodes(original_node)
    quantized_bias_add_node = create_node("QuantizedBiasAdd",
                                          quantized_bias_add_name,
                                          all_input_names)
    set_attr_dtype(quantized_bias_add_node, "T1", dtypes.quint8)
    set_attr_dtype(quantized_bias_add_node, "T2", dtypes.quint8)
    set_attr_dtype(quantized_bias_add_node, "out_type", dtypes.qint32)
    self.add_output_graph_node(quantized_bias_add_node)
    quantize_down_name = self.add_quantize_down_nodes(original_node,
                                                      quantized_bias_add_name)
    self.add_dequantize_result_node(quantize_down_name, original_node.name)

  def eightbitize_single_input_tensor_node(self, original_node,
                                           add_op_function):
    """Replaces a single-tensor node with the eight bit equivalent sub-graph.

    Converts a node like this:

       Shape(f)   Input(f)
         |          |
         +--------v v
                Operation
                    |
                    v
                   (f)

     Into a quantized equivalent:

                    Input(f)              ReshapeDims
                       +------v v-------------+
                       |    Reshape
                       |      |
                       |      |          ReductionDims
                       |      +-----+         |
                       |      | +---c---------+
                       |      v v   v v-------+
                       |      Min   Max
                       |  +----+      |
                       v  v  v--------+
                      Quantize
                          |
                          v
                   QuantizedOperation
                      |   |   |
                      v   v   v
                      Dequantize
                          |
                          v
                         (f)


    Args:
      original_node: Float node to be converted.
      add_op_function: Function to create the actual node.

    Returns:
      Subgraph representing the quantized version of the original node.

    """
    quantized_op_name = original_node.name + "_eightbit_quantized"
    quantized_op_type = "Quantized" + original_node.op
    all_input_names = self.add_eightbit_prologue_nodes(original_node)
    quantized_op_node = create_node(quantized_op_type, quantized_op_name,
                                    all_input_names)
    add_op_function(original_node, quantized_op_node)
    self.add_output_graph_node(quantized_op_node)
    self.add_dequantize_result_node(quantized_op_name, original_node.name)

  def add_pool_function(self, original_node, quantized_op_node):
    set_attr_dtype(quantized_op_node, "T", dtypes.quint8)
    copy_attr(quantized_op_node, "ksize", original_node.attr["ksize"])
    copy_attr(quantized_op_node, "strides", original_node.attr["strides"])
    copy_attr(quantized_op_node, "padding", original_node.attr["padding"])

  def add_relu_function(self, unused_arg_node, quantized_op_node):
    set_attr_dtype(quantized_op_node, "Tinput", dtypes.quint8)

  def eightbitize_concat_node(self, original_node):
    """Replaces a Concat node with the eight bit equivalent sub-graph.

    Converts a node like this:

       Shape(f)   Input0(f)   Input1(f)
         |          |            |
         +--------v v v----------+
                  Concat
                    |
                    v
                   (f)

     Into a quantized equivalent:

       Shape(f)     Input0(f)             ReshapeDims                  Input1(f)
         |             +------v v--------------+------------------v v------+
         |             |    Reshape                             Reshape    |
         |             |      |                                     |      |
         |             |      |           ReductionDims             |      |
         |             |      +------+         |           +--------+      |
         |             |      |  +---c---------+-----------c-----+  |      |
         |             |      +v v   v v-------+---------v v     v v+      |
         |             |       Min   Max                 Min     Max       |
         |             |  +----+      |                   |       +-----+  |
         |             v  v  v--------+                   +----------v  v  v
         |            Quantize                                       Quantize
         |                +------------------+   +----------------------+
         +-------------------------------+   |   |
                                         v   v   v
                                      QuantizedConcat
                                         |   |   |
                                         v   v   v
                                        Dequantize
                                             |
                                             v
                                            (f)
    Args:
      original_node: Float node to be converted.

    Returns:
      Subgraph representing the quantized version of the original node.

    """
    namespace_prefix = original_node.name + "_eightbit"
    quantized_concat_name = namespace_prefix + "_quantized_concat"
    reshape_dims_name, reduction_dims_name = self.add_common_quantization_nodes(
        namespace_prefix)
    shape_input_name = original_node.input[0]
    original_inputs = original_node.input[1:]
    input_names = []
    min_names = []
    max_names = []
    for original_input_name in original_inputs:
      quantize_input_name, min_input_name, max_input_name = (
          self.eightbitize_input_to_node(namespace_prefix, original_input_name,
                                         reshape_dims_name,
                                         reduction_dims_name))
      input_names.append(quantize_input_name)
      min_names.append(min_input_name)
      max_names.append(max_input_name)
    all_input_names = [shape_input_name]
    all_input_names.extend(input_names)
    all_input_names.extend(min_names)
    all_input_names.extend(max_names)
    quantized_concat_node = create_node("QuantizedConcat",
                                        quantized_concat_name, all_input_names)
    set_attr_int(quantized_concat_node, "N", len(original_inputs))
    set_attr_dtype(quantized_concat_node, "T", dtypes.quint8)
    self.add_output_graph_node(quantized_concat_node)
    self.add_dequantize_result_node(quantized_concat_name, original_node.name)

  def eightbitize_placeholder_node(self, current_node):
    """Replaces a placeholder node with a quint8 placeholder node+dequantize."""
    name = current_node.name

    # Convert the placeholder into a quantized type.
    output_node = node_def_pb2.NodeDef()
    output_node.CopyFrom(current_node)
    set_attr_dtype(output_node, "dtype", dtypes.quint8)
    output_node.name += "_original_input"
    self.add_output_graph_node(output_node)

    # Add a dequantize to convert back to float.
    dequantize_node = create_node("Dequantize", name, [
        output_node.name, "quantized_input_min_value",
        "quantized_input_max_value"
    ])
    set_attr_dtype(dequantize_node, "T", dtypes.quint8)
    set_attr_string(dequantize_node, "mode", b"MIN_FIRST")
    self.add_output_graph_node(dequantize_node)

    # For the descent over the graph to work, the dequantize node must be named
    # current_node.name.  However, for the feeding of the graph to work, the
    # placeholder must have the name current_node.name; so record a final set
    # of renames to apply after all processing has been done.
    self.final_node_renames[output_node.name] = name
    self.final_node_renames[dequantize_node.name] = name + "_dequantize"

  def eightbitize_reshape_node(self, original_node):
    """Replaces a Reshape node with the eight bit equivalent sub-graph.

    Args:
      original_node: Float node to be converted.

    Returns:
      Subgraph representing the quantized version of the original node.

    """
    namespace_prefix = original_node.name + "_eightbit"
    quantized_reshape_name = namespace_prefix + "_quantized_reshape"
    reshape_dims_name, reduction_dims_name = self.add_common_quantization_nodes(
        namespace_prefix)
    shape_input_name = original_node.input[1]
    quantize_input_name, min_input_name, max_input_name = (
        self.eightbitize_input_to_node(namespace_prefix, original_node.input[0],
                                       reshape_dims_name, reduction_dims_name))
    quantized_reshape_node = create_node(
        "QuantizedReshape", quantized_reshape_name,
        [quantize_input_name, shape_input_name, min_input_name, max_input_name])
    set_attr_dtype(quantized_reshape_node, "T", dtypes.quint8)
    self.add_output_graph_node(quantized_reshape_node)
    self.add_dequantize_result_node(quantized_reshape_name, original_node.name)

  def eightbitize_batch_norm_node(self, original_node):
    """Replaces a MatMul node with the eight bit equivalent sub-graph."""
    namespace_prefix = original_node.name + "_eightbit"
    original_input_name = original_node.input[0]
    original_mean_name = original_node.input[1]
    original_variance_name = original_node.input[2]
    original_beta_name = original_node.input[3]
    original_gamma_name = original_node.input[4]
    quantized_batch_norm_name = namespace_prefix + "_quantized_batch_norm"

    reshape_dims_name, reduction_dims_name = self.add_common_quantization_nodes(
        namespace_prefix)
    quantize_input_name, min_input_name, max_input_name = (
        self.eightbitize_input_to_node(namespace_prefix, original_input_name,
                                       reshape_dims_name, reduction_dims_name))
    quantize_mean_name, min_mean_name, max_mean_name = (
        self.eightbitize_input_to_node(namespace_prefix, original_mean_name,
                                       reshape_dims_name, reduction_dims_name))
    quantize_variance_name, min_variance_name, max_variance_name = (
        self.eightbitize_input_to_node(namespace_prefix, original_variance_name,
                                       reshape_dims_name, reduction_dims_name))
    quantize_beta_name, min_beta_name, max_beta_name = (
        self.eightbitize_input_to_node(namespace_prefix, original_beta_name,
                                       reshape_dims_name, reduction_dims_name))
    quantize_gamma_name, min_gamma_name, max_gamma_name = (
        self.eightbitize_input_to_node(namespace_prefix, original_gamma_name,
                                       reshape_dims_name, reduction_dims_name))
    quantized_batch_norm_node = create_node(
        "QuantizedBatchNormWithGlobalNormalization", quantized_batch_norm_name,
        [
            quantize_input_name, min_input_name, max_input_name,
            quantize_mean_name, min_mean_name, max_mean_name,
            quantize_variance_name, min_variance_name, max_variance_name,
            quantize_beta_name, min_beta_name, max_beta_name,
            quantize_gamma_name, min_gamma_name, max_gamma_name
        ])
    set_attr_dtype(quantized_batch_norm_node, "Tinput", dtypes.quint8)
    set_attr_dtype(quantized_batch_norm_node, "out_type", dtypes.qint32)
    copy_attr(quantized_batch_norm_node, "scale_after_normalization",
              original_node.attr["scale_after_normalization"])
    copy_attr(quantized_batch_norm_node, "variance_epsilon",
              original_node.attr["variance_epsilon"])
    self.add_output_graph_node(quantized_batch_norm_node)
    quantize_down_name = self.add_quantize_down_nodes(original_node,
                                                      quantized_batch_norm_name)
    self.add_dequantize_result_node(quantize_down_name, original_node.name)

  def add_output_graph_node(self, output_node):
    """Inserts one node into the new graph."""
    self.output_graph.node.extend([output_node])

  def remove_redundant_quantization(self, old_graph):
    """Removes unneeded pairs of quantize/dequantize ops from the graph.

    This is a bit of a tricky function, because it's attempting to spot the
    pattern of dequantizing from eight-bit up to float, and then immediately
    quantizing back down to eight bits again, that's introduced by previous
    passes that do 'key-hole' conversions of individual nodes but have to
    convert back to float to match the previous output interface, since they
    don't know that the next op can handle quantized tensors.
    It works by:
     - Looking for Quantize nodes.
     - Checking to see if their first input is a Dequantize node.
     - Seeing if their min/max inputs come from Min/Max nodes.
     - Making sure those Min/Max nodes are being fed from the same Dequantize.
     - Or that the Min is indirectly being fed from the same Dequantize as Max.
     - Making sure the Dequantize is going through a Reshape (which we add
       during the previous pass when we create the quantize sub-graph).
     - Looking for the dims Const op for the Min/Max dims.
    If all of these conditions are met, then it's a sub-graph pattern that
    we know how to optimize out (and is likely the common one we've introduced).
    We then rewire the graph to skip it entirely, and then rely on the dead node
    removal pass to get rid of any nodes that are no longer needed.

    Args:
      old_graph: The model we'll be stripping redundant nodes from.

    Returns:
      A graph with the unnecessary nodes removed.

    Raises:
      ValueError: Two nodes with the same name were found in the graph.
    """
    old_nodes_map = self.create_nodes_map(old_graph)
    self.output_graph = graph_pb2.GraphDef()
    inputs_to_rename = {}
    # We go through all the nodes, looking for any that match the patterns we
    # know how to optimize away.
    for node in old_graph.node:
      # We always start with a Quantize node, and examine its inputs to see if
      # they are in a form that can be removed.
      if node.op not in ["Quantize", "QuantizeV2"]:
        continue
      dequantize_node_name = node_name_from_input(node.input[0])
      if dequantize_node_name not in old_nodes_map:
        raise ValueError("Input node name '" + dequantize_node_name +
                         "' not found in node '" + node.name + "'")
      dequantize_node = old_nodes_map[dequantize_node_name]
      # Do we have a Dequantize feeding in, with the same type as the Quantize?
      if dequantize_node.op != "Dequantize":
        continue
      if node.attr["T"] != dequantize_node.attr["T"]:
        continue
      # Now look at the other inputs, and ensure they're Min/Max nodes.
      min_node_name = node_name_from_input(node.input[1])
      max_node_name = node_name_from_input(node.input[2])
      min_node = old_nodes_map[min_node_name]
      max_node = old_nodes_map[max_node_name]
      is_min_right_type = (min_node.op in ["Min", "Dequantize"])
      is_max_right_type = (max_node.op in ["Max", "Dequantize"])
      if not is_min_right_type or not is_max_right_type:
        print("Didn't find expected types on inputs : %s, %s." % (min_node.op,
                                                                  max_node.op))
        continue
      min_node_input_name = node_name_from_input(min_node.input[0])
      max_node_input_name = node_name_from_input(max_node.input[0])
      # There are two different patterns for Min nodes we can recognize, one
      # where the input comes directly from the same one as the Max, and
      # another where we run it through another Min first, so check for both.
      is_same_input = False
      if min_node_input_name == max_node_input_name:
        is_same_input = True
      else:
        first_min_node_input = old_nodes_map[min_node_input_name]
        if first_min_node_input.op == "Concat":
          second_min_node_name = node_name_from_input(
              first_min_node_input.input[1])
          second_min_node = old_nodes_map[second_min_node_name]
          if second_min_node.op == "Min":
            second_min_node_input_name = node_name_from_input(
                second_min_node.input[0])
            is_same_input = (second_min_node_input_name == max_node_input_name)
      if not is_same_input:
        print("Different min/max inputs: " + min_node_input_name)
        continue
      # We recognize this pattern, so mark the graph edges to be rewired to
      # route around it entirely, since we know it's a no-op.
      dequantize_source_name = node_name_from_input(dequantize_node.input[0])
      node_tensor_name = ensure_tensor_name_has_port(node.name)
      min_tensor_name = node.name + ":1"
      max_tensor_name = node.name + ":2"
      inputs_to_rename[node_tensor_name] = dequantize_source_name
      inputs_to_rename[min_tensor_name] = dequantize_node.input[1]
      inputs_to_rename[max_tensor_name] = dequantize_node.input[2]
    # Finally we apply all the rewiring we've marked to the graph.
    for node in old_graph.node:
      for index, input_full_name in enumerate(node.input):
        input_name = ensure_tensor_name_has_port(input_full_name)
        if input_name in inputs_to_rename:
          node.input[index] = inputs_to_rename[input_name]
      self.add_output_graph_node(node)
    return self.output_graph

  def apply_final_node_renames(self):
    """Applies node renames in self.final_node_renames to self.output_graph."""
    old_graph = self.output_graph
    self.output_graph = graph_pb2.GraphDef()
    for node in old_graph.node:
      node.name = self.final_node_renames.get(node.name, node.name)
      for index, input_name in enumerate(node.input):
        node_name = node_name_from_input(input_name)
        input_full_name = ensure_tensor_name_has_port(input_name)
        if node_name in self.final_node_renames:
          node.input[index] = "%s%s" % (self.final_node_renames[node_name],
                                        input_full_name[len(node_name):])
      self.add_output_graph_node(node)
    return self.output_graph

  def remove_dead_nodes(self, output_names):
    """Removes nodes that are no longer needed for inference from the graph."""
    old_output_graph = self.output_graph
    self.output_graph = graph_util.extract_sub_graph(old_output_graph,
                                                     output_names)

  def quantize_weights(self, input_graph, quantization_mode):
    """Quantize float Const ops.

    There are two modes of operations, both replace float Const ops with
    quantized values.
    1. If quantization_mode is "weights_rounded", this function replaces float
    Const ops with quantized float Const ops - same as the original op, but
    float values being mapped to the center of one of 1<<FLAGS.bitdepth buckets.
    This does not change the raw model size, but compression algorithms such as
    zip (as used for compressing apks) or bzip2 will achieve a very good
    compression ratio.
    2. For other quantization modes ("MIN_COMBINED" or "MIN_FIRST"), float
    Const ops are quantized and replaced by a tuple of four ops to perform
    the dequantization at runtime:
    * eight-bit Const (bucket indices, same shape as original float Const op
    * two float Const ops (min and max value of original float Const op)
    * Dequantize op to convert the eight-bit consts to float tensors.
    The quantization mode is important because we see accuracy problems when
    quantizing weights for different situations depending on the algorithm
    used. We haven't figured out exactly what the underlying cause is yet,
    unfortunately.

    Args:
      input_graph: A GraphDef of the model containing float Const ops.
      quantization_mode: How to quantize and dequantize the values.

    Returns:
      A GraphDef of the converted graph.

    Raises:
      ValueError: If quantization_mode is unsupported.
    """
    output_graph = graph_pb2.GraphDef()
    for input_node in input_graph.node:
      should_quantize = False
      if input_node.op == "Const":
        dtype = dtypes.as_dtype(input_node.attr["dtype"].type)
        if dtype == dtypes.float32:
          should_quantize = True
      if should_quantize:
        if quantization_mode == "weights_rounded":
          output_graph.node.extend(quantize_weight_rounded(input_node))
        elif quantization_mode in (b"MIN_COMBINED", b"MIN_FIRST"):
          output_graph.node.extend(
              quantize_weight_eightbit(input_node, quantization_mode))
        else:
          raise ValueError("Unsupported quantization mode %s." %
                           quantization_mode)
      else:
        output_node = node_def_pb2.NodeDef()
        output_node.CopyFrom(input_node)
        output_graph.node.extend([output_node])
    return output_graph

  def set_input_graph(self, new_input_graph):
    self.input_graph = new_input_graph
    self.nodes_map = self.create_nodes_map(self.input_graph)


def main(unused_args):
  if not gfile.Exists(FLAGS.input):
    print("Input graph file '" + FLAGS.input + "' does not exist!")
    return -1

  known_modes = [
      "round", "quantize", "eightbit", "weights", "test", "weights_rounded"
  ]
  if not any(FLAGS.mode in s for s in known_modes):
    print("mode is '" + FLAGS.mode + "', not in " + ", ".join(known_modes) +
          ".")
    return -1

  tf_graph = graph_pb2.GraphDef() # đọc file chứ graph
  with gfile.Open(FLAGS.input, "rb") as f:
    data = f.read()
    tf_graph.ParseFromString(data)

  graph = ops.Graph() # import vào mt TF để thao tác
  with graph.as_default():
    importer.import_graph_def(tf_graph, input_map={}, name="")

  quantized_input_range = None # thiết lập min, max của input
  if FLAGS.quantized_input:
    quantized_input_range = [
        FLAGS.quantized_input_min, FLAGS.quantized_input_max
    ]

  fallback_quantization_range = None  # Trường hợp đồ thị thiếu thông tin min/max cho một số layer Dùng giá trị fallback để tránh lỗi, nhưng độ chính xác có thể giảm
  if (FLAGS.quantized_fallback_min is not None or
      FLAGS.quantized_fallback_max is not None):
    assert FLAGS.quantized_fallback_min is not None
    assert FLAGS.quantized_fallback_max is not None
    fallback_quantization_range = [
        FLAGS.quantized_fallback_min, FLAGS.quantized_fallback_max
    ]

  rewriter = GraphRewriter(tf_graph, FLAGS.mode, quantized_input_range,
                           fallback_quantization_range)

  output_graph = rewriter.rewrite(FLAGS.output_node_names.split(",")) # gọi rewrite() để lượng tử hóa đồ thị

  f = gfile.FastGFile(FLAGS.output, "wb")
  f.write(output_graph.SerializeToString())

  return 0


if __name__ == "__main__":
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
  app.run()

## graph_pb2tb.py
Hiển thị cấu trúc mô hình trên TensorBoard

In [ ]:
def load_graph(graph_pb_path):
    with open(graph_pb_path, 'rb') as f:
        content = f.read()
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(content)
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
    return graph

def graph_to_tensorboard(graph, out_dir):
    with tf.Session():  # Mở session
        train_writer = tf.summary.FileWriter(out_dir)  # Tạo đối tượng ghi log vào out_dir
        train_writer.add_graph(graph)  # Ghi cấu trúc đồ thị

def main(out_dir, graph_pb_path):
    graph = load_graph(graph_pb_path)
    graph_to_tensorboard(graph, out_dir)  # Xuất đồ thị ra TensorBoard

# if __name__ == "__main__":
#   os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#   main(*sys.argv[1:])